# ET Blue Visualization

This notebook analyzes irrigated area, irrigable area (irrigated at least once in 7 years), and irrigation volumes per 5km grid.

Key analysis includes:
- Statistics by Gemeinde
- Statistics by crop class
- Statistics by month (e.g. time-series of vol within Bibertal areas with licence)
- Fraction of irrigated area inside and ausserhalb BewässerungsflächenSonstige, within Perimeter
- Fraction of area within perimeter and outside perimeter, in Kanton Schaffhausen
- Visualize ETgreen distribution in the three cantons (e.g., average August ETgreen)

In [ ]:
# Import required libraries
import ee
import geemap
import pandas as pd
import numpy as np

# Initialize Google Earth Engine
ee.Authenticate()
# ee.Initialize(project="thurgau-irrigation")
ee.Initialize(project="ee-sahellakes")


## Load ET Blue Data for Different Years and Regions

Loading ET blue field statistics for Schaffhausen, Zürich, and Thurgau for years 2018-2024.

In [ ]:
# Define years to process
years = [2018, 2019, 2020, 2021, 2022, 2023, 2024]
regions = ['Schaffhausen', 'Zuerich', 'Thurgau']

# Dictionary to store feature collections for each year
yearly_collections = {}
yearly_images = {}
yearly_volume_images = {}

# Base path for the assets
base_path = 'projects/thurgau-irrigation/assets/ZH_SH_TG/ET_blue_per_field_annual_v2_withETA/ETblue_field_stats_'
suffix = '_ETaETc70_ETblue10'

for year in years:
    # Load feature collections for each region
    fc_sh = ee.FeatureCollection(f'{base_path}Schaffhausen_{year}{suffix}')
    fc_zh = ee.FeatureCollection(f'{base_path}Zuerich_{year}{suffix}')
    fc_tg = ee.FeatureCollection(f'{base_path}Thurgau_{year}{suffix}')
    
    # Merge all regions for this year
    fc_year = fc_zh.merge(fc_tg).merge(fc_sh)
    yearly_collections[year] = fc_year
    
    # Create binary irrigation image (irrigated or not)
    img_binary = fc_year.reduceToImage(['ETblue_mm'], 'first').unmask(0).gt(0)
    yearly_images[year] = img_binary
    
    # Create volume image (average ET blue in mm)
    img_volume = fc_year.reduceToImage(['ETblue_mm'], 'mean').unmask(0)
    yearly_volume_images[year] = img_volume

print(f"Loaded data for {len(years)} years: {years}")

In [ ]:
# # Create interactive map
# Map = geemap.Map()
# Map
# # Add some layers to the map for visualization
# Map.addLayer(
#     yearly_images[2018].updateMask(yearly_images[2018].gt(0)),
#     {'min': 0, 'max': 1, 'palette': ['white', 'yellow', 'orange', 'red']},
#     '2018 Irrigated Areas',
#     False
# )

# Map.addLayer(
#     yearly_images[2023].updateMask(yearly_images[2023].gt(0)),
#     {'min': 0, 'max': 1, 'palette': ['white', 'yellow', 'orange', 'red']},
#     '2023 Irrigated Areas',
#     False
# )

## Calculate Irrigation Frequency and Average Volumes

In [ ]:
# Calculate irrigation frequency (how many years irrigated)
irrigation_frequency = ee.Image(0)
total_volume = ee.Image(0)

for year in years:
    irrigation_frequency = irrigation_frequency.add(yearly_images[year])
    total_volume = total_volume.add(yearly_volume_images[year])

# Average volume over the 7-year period
average_volume = total_volume.divide(7)

print("Calculated irrigation frequency and average volumes")

In [ ]:
# Add this at the beginning of cells that might produce duplicate output
from IPython.display import clear_output
clear_output(wait=True)

# Combine all years into a single feature collection
fc_all = ee.FeatureCollection([yearly_collections[year] for year in years]).flatten()

print('Total features in combined collection:', fc_all.size().getInfo())
print('First feature:', fc_all.first().getInfo())
#properties of the first feature
print('Properties of the first feature:', fc_all.first().propertyNames().getInfo())


## Vegetable Irrigation Analysis

In [ ]:
# Vegetable irrigation (class 2)
vegetable_irrigation = (
    fc_all.filter(ee.Filter.eq('class', 1))
    .reduceToImage(['ETblue_mm'], 'sum')
    .unmask(0)
    .divide(7)
)

print('Vegetable irrigation image created')

In [ ]:
# Map = geemap.Map()
# Map
# # Add irrigation frequency and volume layers
# Map.addLayer(
#     irrigation_frequency.updateMask(irrigation_frequency.gt(0)),
#     {'min': 1, 'max': 4, 'palette': ['white', 'yellow', 'orange', 'red']},
#     'Irrigation Frequency'
# )

# Map.addLayer(
#     average_volume.updateMask(average_volume.gt(0)),
#     {'min': 3, 'max': 20, 'palette': ['white', 'yellow', 'orange', 'red']},
#     'Average ET Blue Volume'
# )

## Load Administrative Boundaries and Special Areas

In [ ]:
# Load perimeter and special areas
perimeter = ee.FeatureCollection('projects/thurgau-irrigation/assets/Schaffhausen/perimeter_bew_oberflaechengew')
bewaesserungsflaechen = ee.FeatureCollection('projects/thurgau-irrigation/assets/Schaffhausen/Bewaesserungsflaechen_Bibertal')
adm1_units = ee.FeatureCollection('projects/thurgau-irrigation/assets/GIS/Kantone_simplified100m').filter(ee.Filter.Or(ee.Filter.eq('NAME','Schaffhausen'),ee.Filter.eq('NAME','Thurgau'),ee.Filter.eq('NAME','Zürich')))
gemeinden = ee.FeatureCollection('projects/thurgau-irrigation/assets/GIS/Gemeinden_Schweiz')

# # Add perimeter visualization
# perimeter_viz = ee.Image().paint(perimeter, 0, 2)
# Map.addLayer(
#     perimeter_viz.visualize({'palette': ['black'], 'forceRgbOutput': True}),
#     {},
#     'Perimeter',
#     False
# )

# # Add Bewässerungsflächen visualization
# bewaesserungsflaechen_viz = ee.Image().paint(bewaesserungsflaechen, 0, 1)
# Map.addLayer(
#     bewaesserungsflaechen_viz.visualize({'palette': ['green'], 'forceRgbOutput': True}),
#     {},
#     'Bewässerungsflächen Bibertal'
# )

In [ ]:
##Lets refine the crop classes of Class 1 (vegetable and potato) into more specific categories
vegetables_list = ee.List([
    "Einjährige Freilandgemüse, ohne Konservengemüse",
    "Mehrjährige gärtnerische Freilandkulturen (nicht im Gewächshaus)",
    "Mehrjährige gärtnerische Freilandkulturen",
    "Einjährige Freilandgemüse o. Konservengemüse",
    "Freiland-Konservengemüse", 
    "Freilandgemüse (ohne Kons.gemüse)",
    "Spargel",
    "Ölkürbisse",
    "Oelkürbisse",
    "Rhabarber",
    "Einjährige gärtnerische Freilandkulturen (Blumen, Rollrasen usw.)",
    "Einjä. gärtn. Freilandkult.(Blumen,Rollrasen)",
    "Mehrjährige Gewürz- und Medizinalpflanzen"
])

kartoffel_list = ee.List([
    "Kartoffeln",
    "Pflanzkartoffeln (Vertragsanbau)"
])

beeren_list = ee.List([
    "Einjährige Beeren (z.B. Erdbeeren)","Einjährige Beeren (Erdbeeren etc.)",
    "Mehrjährige Beeren"
])

others_list = ee.List([
    "Soja",
    "Tabak"
])

# Function to reclassify Class 1 into more specific categories
def reclassify_crop(feature):
    crop_name = ee.String(feature.get('nutzung'))
    new_class = feature.get('class')  # Default to original class
    
    # Only reclassify if it's class 1 (vegetables/potatoes)
    new_class = ee.Algorithms.If(
        ee.Number(feature.get('class')).eq(1),
        ee.Algorithms.If(vegetables_list.contains(crop_name), 11,
        ee.Algorithms.If(kartoffel_list.contains(crop_name), 12,
        ee.Algorithms.If(beeren_list.contains(crop_name), 13,
        ee.Algorithms.If(others_list.contains(crop_name), 14, new_class)))),
        ##lets assume second crops (class 5) are also vegetables and reclassify them to 11
        ee.Algorithms.If(ee.Number(feature.get('class')).eq(5), 11, new_class)
    )
    
    return feature.set('refined_class', new_class)

## Bibertal Analysis

In [ ]:
###ET blue and ET
# Add this at the beginning of cells that might produce duplicate output
from IPython.display import clear_output
clear_output(wait=True)


# Calculate ET blue and ET by administrative regions, month, year, and class

"""
Calculate ET blue volumes for different administrative regions across:
- Years: 2018-2024
- Months: May-September (05-09)  
- Classes: 1-5 (vegetables_and_others, maize, kunstwiesen, zuckerrueben, winter_crops)
- Regions: Inside Perimeter (outside/inside Bewaesserungsflaechen), Cantons (SH/ZH/TG)
"""
def calculate_ET_ETBlue_Bibertal():

    # Define parameters
    years = [2018, 2019, 2020, 2021, 2022, 2023, 2024]
    months = ['05', '06', '07', '08', '09']
    classes = {
        2: 'Mais',
        3: 'Grünland',
        4: 'Zuckerrüben',
        11: 'Gemüse',
        12: 'Kartoffeln', 
        13: 'Beeren',
        14: 'Sonstige'
    }

    # Define administrative regions
    regions = {
        'Inside_Perimeter': {
            'name': 'Inside Perimeter',
            'description': 'Fields within perimeter'
        },
    }

    # Results storage
    results = []

    # Base path for the assets
    base_path = 'projects/thurgau-irrigation/assets/ZH_SH_TG/ET_blue_per_field_annual_v2_withETA/ETblue_field_stats_'
    suffix = '_ETaETc70_ETblue10'

    for year in years:

        # Load feature collections for each region
        fc_sh = ee.FeatureCollection(f'{base_path}Schaffhausen_{year}{suffix}')
        fc_zh = ee.FeatureCollection(f'{base_path}Zuerich_{year}{suffix}')
        fc_tg = ee.FeatureCollection(f'{base_path}Thurgau_{year}{suffix}')
        
        print(f"\nProcessing year {year}:")
        
        # Get feature collection for this year
        fc_year = fc_zh.merge(fc_tg).merge(fc_sh).map(reclassify_crop)
        
        for region_code, region_info in regions.items():
            print(f"  Processing {region_info['name']}...")
            
            # Filter by region
            if region_code == 'Inside_Perimeter':
                # Inside perimeter
                fc_region = (fc_year
                            .filterBounds(perimeter.geometry()))
                
            for class_num, class_name in classes.items():
                # Filter by crop class
                fc_class = fc_region.filter(ee.Filter.eq('refined_class', class_num))
                
                for month in months:
                    # Property name for this month's ET blue
                    etblue_prop = f'ETblue_{month}'
                    et_prop = f'ETA_{month}'
                    
                    # Calculate volume: ET_BLUE_MM * AREA_M2 / 1000 to get m3
                    def calculate_field_volume(feature):
                        etblue_mm = ee.Number(feature.get(etblue_prop)).max(0)  # Ensure non-negative
                        et_mm = ee.Number(feature.get(et_prop)).max(0)  # Ensure non-negative
                        area_m2 = ee.Number(feature.get('flaeche_m2'))
                        volume_m3 = etblue_mm.multiply(area_m2).divide(1000)
                        volume_m3_ET = et_mm.multiply(area_m2).divide(1000).multiply(etblue_mm.gt(0))
                        return feature.set('volume_m3', volume_m3).set('volume_m3_ET', volume_m3_ET)

                    # Apply calculation and sum volumes
                    fc_with_volumes = fc_class.map(calculate_field_volume)
                    total_volume = fc_with_volumes.aggregate_sum('volume_m3')
                    total_volume_ET = fc_with_volumes.aggregate_sum('volume_m3_ET')
                    field_count = fc_class.size()
                    
                    # Get the actual values
                    try:
                        volume_val = total_volume.getInfo()
                        count_val = field_count.getInfo()
                        volume_val_ET = total_volume_ET.getInfo()

                        # Calculate total irrigated area for this month/year/class/region
                        fc_irrigated = fc_class.filter(ee.Filter.gt(etblue_prop, 0))
                        total_area = fc_irrigated.aggregate_sum('flaeche_m2')
                        area_val = total_area.getInfo() if total_area.getInfo() is not None else 0

                        volume_mm3_ETblue = volume_val / area_val * 1000 if area_val > 0 else 0
                        volume_mm3_ET = volume_val_ET / area_val * 1000 if area_val > 0 else 0

                        results.append({
                            'year': year,
                            'month': month,
                            'class': class_num,
                            'class_name': class_name,
                            'region_code': region_code,
                            'region_name': region_info['name'],
                            'mm_ETblue': volume_mm3_ETblue,
                            'mm_ET': volume_mm3_ET,
                            'field_count': count_val,
                            'area_m2': area_val,
                            'area_ha': area_val / 10000  # Convert to hectares
                        })
                        
                        if volume_val > 0:  # Only print non-zero results for first region to avoid spam
                            # if region_code == 'Inside_Perimeter_Outside_Bewaesserungsflaechen':
                            print(f"    {year}-{month} | Class {class_num} ({class_name}): "
                                    f"{volume_mm3_ETblue:,.0f} mm and {volume_mm3_ET:,.0f} mm from {count_val} fields")
                            
                    except Exception as e:
                        print(f"    Error calculating {year}-{month} class {class_num} in {region_info['name']}: {e}")
                        results.append({
                            'year': year,
                            'month': month,
                            'class': class_num,
                            'class_name': class_name,
                            'region_code': region_code,
                            'region_name': region_info['name'],
                            'mm_ETblue': 0,
                            'mm_ET': 0,
                            'field_count': 0,
                            'area_m2': 0,
                            'area_ha': 0
                        })
    
    return results

# Execute the calculation
Bibertal_results = calculate_ET_ETBlue_Bibertal()

# Convert to DataFrame for easier analysis
df_Bibertal = pd.DataFrame(Bibertal_results)


In [ ]:
# # Save results to CSV (../data/)
output_csv = '../data/processed/et_blue_mm_Bibertal_v2.csv'
# df_Bibertal.to_csv(output_csv, index=False)

# # load the CSV to check
df_Bibertal = pd.read_csv(output_csv)


In [ ]:
# Add this at the beginning of cells that might produce duplicate output
from IPython.display import clear_output
clear_output(wait=True)

# Calculate volume in Bibertal area for 2018
vol_bib = (
    yearly_collections[2018]
    .filterBounds(perimeter.geometry())
    .aggregate_array('ETblue_vol')
    .reduce(ee.Reducer.sum())
)

print('Volume in Bibertal 2018:', vol_bib.getInfo())


# Calculate ET blue volumes by administrative regions, month, year, and class
def calculate_regional_volumes():
    """
    Calculate ET blue volumes for different administrative regions across:
    - Years: 2018-2024
    - Months: May-September (05-09)  
    - Classes: 1-5 (vegetables_and_others, maize, kunstwiesen, zuckerrueben, winter_crops)
    - Regions: Inside Perimeter (outside/inside Bewaesserungsflaechen), Cantons (SH/ZH/TG)
    """
    
    # Define parameters
    years = [2018, 2019, 2020, 2021, 2022, 2023, 2024]
    months = ['05', '06', '07', '08', '09']
    classes = {
        2: 'Mais',
        3: 'Grünland',
        4: 'Zuckerrüben',
        11: 'Gemüse',
        12: 'Kartoffeln', 
        13: 'Beeren',
        14: 'Sonstige'
    }
    
    # Define administrative regions
    regions = {
        'Inside_Perimeter_Outside_Bewaesserungsflaechen': {
            'name': 'Innerhalb Perimeter, ausserhalb Bewässerungsflächen',
            'description': 'Felder innerhalb des Perimeters aber außerhalb der ausgewiesenen Bewässerungsgebiete'
        },
        'Inside_Perimeter_Inside_Bewaesserungsflaechen': {
            'name': 'Innerhalb Perimeter, innerhalb Bewässerungsflächen', 
            'description': 'Felder innerhalb des Perimeters und innerhalb der ausgewiesenen Bewässerungsgebiete'
        },
        'Canton_Schaffhausen': {
            'name': 'Kanton Schaffhausen',
            'description': 'Alle Felder im Kanton Schaffhausen'
        },
        'Canton_Zuerich': {
            'name': 'Kanton Zürich',
            'description': 'Alle Felder im Kanton Zürich'
        },
        'Canton_Thurgau': {
            'name': 'Kanton Thurgau',
            'description': 'Alle Felder im Kanton Thurgau'
        }
    }
    
    # Results storage
    results = []
    
    print("Calculating ET blue volumes by administrative regions...")
    print("=" * 70)
    
    for year in years:
        print(f"\nProcessing year {year}:")
        
        # Get feature collection for this year
        fc_year = yearly_collections[year].map(reclassify_crop)
        
        for region_code, region_info in regions.items():
            print(f"  Processing {region_info['name']}...")
            
            # Filter by region
            if region_code == 'Inside_Perimeter_Outside_Bewaesserungsflaechen':
                # Inside perimeter but outside Bewässerungsflächen
                fc_region = (fc_year
                            .filterBounds(perimeter.geometry())
                            .filter(ee.Filter.Not(ee.Filter.bounds(bewaesserungsflaechen.geometry()))))
                
            elif region_code == 'Inside_Perimeter_Inside_Bewaesserungsflaechen':
                # Inside perimeter and inside Bewässerungsflächen
                fc_region = (fc_year
                            .filterBounds(perimeter.geometry())
                            .filterBounds(bewaesserungsflaechen.geometry()))
                
            elif region_code == 'Canton_Schaffhausen':
                # Canton Schaffhausen
                canton_sh = adm1_units.filter(ee.Filter.eq('NAME', 'Schaffhausen'))
                fc_region = fc_year.filterBounds(canton_sh.geometry())
                
            elif region_code == 'Canton_Zuerich':
                # Canton Zürich
                canton_zh = adm1_units.filter(ee.Filter.eq('NAME', 'Zürich'))
                fc_region = fc_year.filterBounds(canton_zh.geometry())
                
            elif region_code == 'Canton_Thurgau':
                # Canton Thurgau
                canton_tg = adm1_units.filter(ee.Filter.eq('NAME', 'Thurgau'))
                fc_region = fc_year.filterBounds(canton_tg.geometry())
            
            for class_num, class_name in classes.items():
                # Filter by crop class
                fc_class = fc_region.filter(ee.Filter.eq('refined_class', class_num))
                
                for month in months:
                    # Property name for this month's ET blue
                    et_prop = f'ETblue_{month}'
                    
                    # Calculate volume: ET_BLUE_MM * AREA_M2 / 1000 to get m3
                    def calculate_field_volume(feature):
                        et_mm = ee.Number(feature.get(et_prop)).max(0)  # Ensure non-negative
                        area_m2 = ee.Number(feature.get('flaeche_m2'))
                        volume_m3 = et_mm.multiply(area_m2).divide(1000)
                        return feature.set('volume_m3', volume_m3)
                    
                    # Apply calculation and sum volumes
                    fc_with_volumes = fc_class.map(calculate_field_volume)
                    total_volume = fc_with_volumes.aggregate_sum('volume_m3')
                    field_count = fc_class.size()
                    
                    # Get the actual values
                    try:
                        volume_val = total_volume.getInfo()
                        count_val = field_count.getInfo()
                        
                        # Calculate total irrigated area for this month/year/class/region
                        fc_irrigated = fc_class.filter(ee.Filter.gt(et_prop, 0))
                        total_area = fc_irrigated.aggregate_sum('flaeche_m2')
                        area_val = total_area.getInfo() if total_area.getInfo() is not None else 0
                        
                        results.append({
                            'year': year,
                            'month': month,
                            'class': class_num,
                            'class_name': class_name,
                            'region_code': region_code,
                            'region_name': region_info['name'],
                            'volume_m3': volume_val,
                            'field_count': count_val,
                            'area_m2': area_val,
                            'area_ha': area_val / 10000  # Convert to hectares
                        })
                        
                        if volume_val > 0:  # Only print non-zero results for first region to avoid spam
                            # if region_code == 'Inside_Perimeter_Outside_Bewaesserungsflaechen':
                            print(f"    {year}-{month} | Class {class_num} ({class_name}): "
                                    f"{volume_val:,.0f} m³ from {count_val} fields")
                            
                    except Exception as e:
                        print(f"    Error calculating {year}-{month} class {class_num} in {region_info['name']}: {e}")
                        results.append({
                            'year': year,
                            'month': month,
                            'class': class_num,
                            'class_name': class_name,
                            'region_code': region_code,
                            'region_name': region_info['name'],
                            'volume_m3': 0,
                            'field_count': 0,
                            'area_m2': 0,
                            'area_ha': 0
                        })
    
    return results

# Execute the calculation
regional_results = calculate_regional_volumes()

# Convert to DataFrame for easier analysis
df_regional = pd.DataFrame(regional_results)


In [ ]:
# # Save results to CSV (../data/)
output_csv = '../data/processed/et_blue_volumes_by_region_v2.csv'
# df_regional.to_csv(output_csv, index=False)

# # load the CSV to check
df_regional = pd.read_csv(output_csv)


In [ ]:

print("\n" + "=" * 80)
print("REGIONAL SUMMARY STATISTICS")
print("=" * 80)

# Summary by region
regional_summary = df_regional.groupby(['region_code', 'region_name'])[['volume_m3', 'area_ha']].sum()
print("\nTotal volumes and areas by administrative region:")
for (region_code, region_name), data in regional_summary.iterrows():
    print(f"{region_name}:")
    print(f"  Volume: {data['volume_m3']:,.0f} m³")
    print(f"  Area: {data['area_ha']:,.1f} ha")

# Summary by year and region
print("\n" + "=" * 60)
print("VOLUMES BY YEAR AND REGION")
print("=" * 60)
yearly_regional_summary = df_regional.groupby(['year', 'region_name'])['volume_m3'].sum().unstack(fill_value=0)
print("\nTotal volumes by year and region (m³):")
print(yearly_regional_summary.round(0))

# Summary by class and region
print("\n" + "=" * 60)
print("VOLUMES BY CROP CLASS AND REGION")
print("=" * 60)
class_regional_summary = df_regional.groupby(['class_name', 'region_name'])['volume_m3'].sum().unstack(fill_value=0)
print("\nTotal volumes by crop class and region (m³):")
print(class_regional_summary.round(0))

# Summary by month and region
print("\n" + "=" * 60)
print("VOLUMES BY MONTH AND REGION")
print("=" * 60)
month_regional_summary = df_regional.groupby(['month', 'region_name'])['volume_m3'].sum().unstack(fill_value=0)
month_names = {'05': 'May', '06': 'June', '07': 'July', '08': 'August', '09': 'September', 
               5: 'May', 6: 'June', 7: 'July', 8: 'August', 9: 'September'}
month_regional_summary.index = [month_names[month] for month in month_regional_summary.index]
print("\nTotal volumes by month and region (m³):")
print(month_regional_summary.round(0))

# Grand totals by region
print("\n" + "=" * 60)
print("GRAND TOTALS BY REGION (2018-2024, May-Sep)")
print("=" * 60)
for region_name in df_regional['region_name'].unique():
    region_data = df_regional[df_regional['region_name'] == region_name]
    total_volume = region_data['volume_m3'].sum()
    total_area = region_data['area_ha'].sum()
    print(f"{region_name}:")
    print(f"  Total volume: {total_volume:,.0f} m³")
    print(f"  Total irrigated area: {total_area:,.1f} ha")

# Create pivot tables for better visualization
print("\n" + "=" * 80)
print("DETAILED ANALYSIS - PIVOT TABLES BY REGION")
print("=" * 80)

# For each region, create detailed pivot tables
for region_name in df_regional['region_name'].unique():
    print(f"\n{'='*60}")
    print(f"REGION: {region_name}")
    print(f"{'='*60}")
    
    region_data = df_regional[df_regional['region_name'] == region_name]
    
    # Volume by year and class for this region
    pivot_year_class = region_data.pivot_table(
        values='volume_m3', 
        index='year', 
        columns='class_name', 
        aggfunc='sum', 
        fill_value=0
    )
    print(f"\nVolumes by year and crop class (m³) - {region_name}:")
    print(pivot_year_class.round(0))

    # Volume by year and month for this region
    pivot_year_month = region_data.pivot_table(
        values='volume_m3', 
        index='year', 
        columns='month', 
        aggfunc='sum', 
        fill_value=0
    )
    if not pivot_year_month.empty:
        pivot_year_month.columns = ['May', 'June', 'July', 'August', 'September']
        print(f"\nVolumes by year and month (m³) - {region_name}:")
        print(pivot_year_month.round(0))

    # Area by year and class for this region
    pivot_area_class = region_data.pivot_table(
        values='area_ha', 
        index='year', 
        columns='class_name', 
        aggfunc='sum', 
        fill_value=0
    )
    print(f"\nIrrigated areas by year and crop class (ha) - {region_name}:")
    print(pivot_area_class.round(1))

    # Field count by year and class for this region
    pivot_fields = region_data.pivot_table(
        values='field_count', 
        index='year', 
        columns='class_name', 
        aggfunc='sum', 
        fill_value=0
    )
    print(f"\nNumber of irrigated fields by year and crop class - {region_name}:")
    print(pivot_fields)

# Overall comparison table
print("\n" + "=" * 80)
print("COMPARISON ACROSS ALL REGIONS")
print("=" * 80)

# Total volumes by region and year
comparison_volume = df_regional.groupby(['region_name', 'year'])['volume_m3'].sum().unstack(fill_value=0)
print("\nTotal volumes by region and year (m³):")
print(comparison_volume.round(0))

# Total areas by region and year  
comparison_area = df_regional.groupby(['region_name', 'year'])['area_ha'].sum().unstack(fill_value=0)
print("\nTotal irrigated areas by region and year (ha):")
print(comparison_area.round(1))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import locale

# Set style for better visualization
plt.style.use('seaborn-v0_8')
sns.set_palette("Set2")

# Set German locale for number formatting (fallback to system default if German not available)
try:
    locale.setlocale(locale.LC_NUMERIC, 'de_CH.UTF-8')  # Swiss German
except locale.Error:
    try:
        locale.setlocale(locale.LC_NUMERIC, 'de_DE.UTF-8')  # German
    except locale.Error:
        pass  # Use system default

# Create a copy of df_regional to merge Beeren and Sonstige
df_regional_merged = df_regional.copy()

# Merge Beeren and Sonstige into "Sonstige"
df_regional_merged.loc[df_regional_merged['class_name'].isin(['Beeren', 'Sonstige']), 'class_name'] = 'Sonstige'

# PLOT 1: Perimeter regions comparison (Inside vs ausserhalb BewässerungsflächenSonstige)
perimeter_regions = [
    'Innerhalb Perimeter, ausserhalb Bewässerungsflächen',
    'Innerhalb Perimeter, innerhalb Bewässerungsflächen'
]

fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Updated class order with merged Sonstige (6 classes instead of 7)
class_order = ['Gemüse', 'Kartoffeln', 'Mais', 'Grünland', 'Zuckerrüben', 'Sonstige']
# Color palette for 6 classes
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']

# Function for German/Swiss number formatting (thousands)
def format_thousands_german(x, p):
    if x >= 1000000:
        return f'{x/1000000:.1f} Mio.'
    elif x >= 1000:
        return f"{x/1000:.0f}'000"
    else:
        return f'{x:.0f}'

# Calculate maximum y-value across both perimeter regions for consistent scaling
max_y_value = 0
perimeter_data_list = []

for i, region_name in enumerate(perimeter_regions):
    region_data = df_regional_merged[df_regional_merged['region_name'] == region_name]
    
    # Create pivot table for this region, aggregating merged classes
    pivot_region = region_data.pivot_table(
        values='volume_m3', 
        index='year', 
        columns='class_name', 
        aggfunc='sum', 
        fill_value=0
    )
    
    # Reorder columns if they exist
    existing_classes = [cls for cls in class_order if cls in pivot_region.columns]
    if existing_classes:
        pivot_region_ordered = pivot_region.reindex(columns=existing_classes, fill_value=0)
        perimeter_data_list.append(pivot_region_ordered)
        
        # Update max y-value for consistent scaling
        current_max = pivot_region_ordered.sum(axis=1).max()
        max_y_value = max(max_y_value, current_max)

# Create the subplots with consistent y-axis
for i, region_name in enumerate(perimeter_regions):
    ax = axes[i]
    
    if i < len(perimeter_data_list):
        pivot_region_ordered = perimeter_data_list[i]
        existing_classes = pivot_region_ordered.columns.tolist()
        
        # Use only the colors for existing classes
        class_colors = [colors[class_order.index(cls)] for cls in existing_classes]
        
        pivot_region_ordered.plot(kind='bar', stacked=True, ax=ax, color=class_colors)
        
        # Customize the plot with German labels
        ax.set_title(f'{region_name}\n(2018-2024)', fontsize=12, fontweight='bold')
        ax.set_xlabel('Jahr', fontsize=11)  # German for 'Year'
        ax.set_ylabel('Volumen (m³)', fontsize=11)  # German for 'Volume'
        
        # Set consistent y-axis limits
        ax.set_ylim(0, max_y_value * 1.1)
        
        # Format y-axis with German/Swiss formatting
        ax.yaxis.set_major_formatter(plt.FuncFormatter(format_thousands_german))
        
        # Rotate x-axis labels
        ax.tick_params(axis='x', rotation=45)
        
        # Add legend only to first plot with German title
        if i == 0:
            ax.legend(title='Kulturart', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
        else:
            ax.legend().set_visible(False)
            
        ax.grid(True, alpha=0.3, axis='y')

plt.suptitle('ET Blue Bewässerungsvolumen: Innerhalb vs. Ausserhalb Bewässerungsflächen\n(Perimeter-Gebiet, 2018-2024)', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# PLOT 2: Three cantons comparison
canton_regions = ['Kanton Schaffhausen', 'Kanton Zürich', 'Kanton Thurgau']

fig, axes = plt.subplots(1, 3, figsize=(20, 8))

# Calculate maximum y-value across all cantons for consistent scaling
max_y_value_cantons = 0
canton_data_list = []

for i, region_name in enumerate(canton_regions):
    region_data = df_regional_merged[df_regional_merged['region_name'] == region_name]
    
    # Create pivot table for this region
    pivot_region = region_data.pivot_table(
        values='volume_m3', 
        index='year', 
        columns='class_name', 
        aggfunc='sum', 
        fill_value=0
    )
    
    # Reorder columns if they exist
    existing_classes = [cls for cls in class_order if cls in pivot_region.columns]
    if existing_classes:
        pivot_region_ordered = pivot_region.reindex(columns=existing_classes, fill_value=0)
        canton_data_list.append(pivot_region_ordered)
        
        # Update max y-value for consistent scaling
        current_max = pivot_region_ordered.sum(axis=1).max()
        max_y_value_cantons = max(max_y_value_cantons, current_max)
    else:
        canton_data_list.append(None)

# Create the subplots with consistent y-axis
for i, region_name in enumerate(canton_regions):
    ax = axes[i]
    
    if canton_data_list[i] is not None:
        pivot_region_ordered = canton_data_list[i]
        existing_classes = pivot_region_ordered.columns.tolist()
        
        # Use only the colors for existing classes
        class_colors = [colors[class_order.index(cls)] for cls in existing_classes]
        
        pivot_region_ordered.plot(kind='bar', stacked=True, ax=ax, color=class_colors)
        
        # Customize the plot with German labels
        ax.set_title(f'{region_name}\n(2018-2024)', fontsize=12, fontweight='bold')
        ax.set_xlabel('Jahr', fontsize=11)  # German for 'Year'
        ax.set_ylabel('Volumen (m³)', fontsize=11)  # German for 'Volume'
        
        # Set consistent y-axis limits
        ax.set_ylim(0, max_y_value_cantons * 1.1)
        
        # Format y-axis with German/Swiss formatting
        ax.yaxis.set_major_formatter(plt.FuncFormatter(format_thousands_german))
        
        # Rotate x-axis labels
        ax.tick_params(axis='x', rotation=45)
        
        # Add legend only to first plot with German title
        if i == 0:
            ax.legend(title='Kulturart', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
        else:
            ax.legend().set_visible(False)
            
        ax.grid(True, alpha=0.3, axis='y')
    else:
        ax.text(0.5, 0.5, 'Keine Bewässerungsdaten', ha='center', va='center', transform=ax.transAxes)  # German
        ax.set_title(f'{region_name}\n(Keine Daten)', fontsize=12)  # German

plt.suptitle('ET Blue Bewässerungsvolumen nach Kanton\n(2018-2024)', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Regional comparison summary using merged data - in German
print("\n" + "=" * 70)
print("REGIONALE VERGLEICHSÜBERSICHT")  # German
print("=" * 70)

for region_name in df_regional_merged['region_name'].unique():
    region_data = df_regional_merged[df_regional_merged['region_name'] == region_name]
    total_volume = region_data['volume_m3'].sum()
    total_area = region_data['area_ha'].sum()
    avg_annual_volume = region_data.groupby('year')['volume_m3'].sum().mean()
    
    print(f"\n{region_name}:")
    print(f"  Gesamtvolumen (2018-2024): {total_volume:,.0f} m³")  # German
    print(f"  Gesamte bewässerte Fläche: {total_area:,.1f} ha")  # German
    print(f"  Durchschnittliches Jahresvolumen: {avg_annual_volume:,.0f} m³")  # German
    
    # Peak year for this region
    yearly_volumes = region_data.groupby('year')['volume_m3'].sum()
    if not yearly_volumes.empty:
        peak_year = yearly_volumes.idxmax()
        peak_volume = yearly_volumes.max()
        print(f"  Spitzenjahr: {peak_year} ({peak_volume:,.0f} m³)")  # German

# Comparison between Perimeter regions
print(f"\n" + "=" * 50)
print("PERIMETER-VERGLEICH")  # German
print("=" * 50)

perimeter_regions = [
    'Innerhalb Perimeter, innerhalb Bewässerungsflächen',
    'Innerhalb Perimeter, ausserhalb Bewässerungsflächen',
]

perimeter_data = df_regional_merged[df_regional_merged['region_name'].isin(perimeter_regions)]
perimeter_summary = perimeter_data.groupby('region_name')[['volume_m3', 'area_ha']].sum()

total_perimeter_volume = perimeter_summary['volume_m3'].sum()
total_perimeter_area = perimeter_summary['area_ha'].sum()

for region_name, data in perimeter_summary.iterrows():
    volume_pct = (data['volume_m3'] / total_perimeter_volume) * 100 if total_perimeter_volume > 0 else 0
    area_pct = (data['area_ha'] / total_perimeter_area) * 100 if total_perimeter_area > 0 else 0
    print(f"{region_name}:")
    print(f"  Volumen: {data['volume_m3']:,.0f} m³ ({volume_pct:.1f}% der Perimeter-Gesamtmenge)")  # German
    print(f"  Fläche: {data['area_ha']:,.1f} ha ({area_pct:.1f}% der Perimeter-Gesamtfläche)")  # German

# Summary by merged class and region
print("\n" + "=" * 60)
print("VOLUMINA NACH ZUSAMMENGEFASSTEN KULTURARTEN UND REGIONEN")  # German
print("=" * 60)
class_regional_summary_merged = df_regional_merged.groupby(['class_name', 'region_name'])['volume_m3'].sum().unstack(fill_value=0)
print("\nGesamtvolumina nach zusammengefassten Kulturarten und Regionen (m³):")  # German
print(class_regional_summary_merged.round(0))

print(f"\n" + "=" * 50)
print("ZUSAMMENGEFASSTE KULTURARTEN-ÜBERSICHT (6 Klassen)")  # German
print("=" * 50)
print("Aktualisierte Visualisierung mit zusammengefassten Klassen:")  # German
print("1. Gemüse (Klasse 11) - Gemüse")
print("2. Kartoffeln (Klasse 12) - Kartoffeln") 
print("3. Mais (Klasse 2) - Mais")
print("4. Grünland (Klasse 3) - Kunstwiesen/Grünland")
print("5. Zuckerrüben (Klasse 4) - Zuckerrüben")
print("6. Sonstige (Klassen 13,14) - Beeren und andere Kulturen")

In [ ]:
# Add grid for better readability
ax.grid(True, alpha=0.3, axis='x')

# Adjust layout
plt.tight_layout()
plt.show()

# Create temporal comparison for Perimeter regions
fig, ax = plt.subplots(figsize=(20, 9))

# Focus on Perimeter regions for monthly analysis - keep original data order
perimeter_regions = [
    'Innerhalb Perimeter, ausserhalb Bewässerungsflächen',
    'Innerhalb Perimeter, innerhalb Bewässerungsflächen'
]

# Create month-year combinations from the data
df_regional['month_year'] = df_regional['year'].astype(str) + '-' + df_regional['month'].astype(str).str.zfill(2)

# Filter for perimeter regions only
df_perimeter = df_regional[df_regional['region_name'].isin(perimeter_regions)]

# Create pivot table for month-year vs region
pivot_month_year_region = df_perimeter.pivot_table(
    values='volume_m3', 
    index='month_year', 
    columns='region_name', 
    aggfunc='sum', 
    fill_value=0
)

# Add April months with zero values for better year separation
years = [2018, 2019, 2020, 2021, 2022, 2023, 2024]

# Create complete month-year index including April
all_month_years = []
for year in years:
    for month in ['04', '05', '06', '07', '08', '09']:
        all_month_years.append(f'{year}-{month}')

# Reindex to include all months (April will have zero values)
pivot_month_year_region_complete = pivot_month_year_region.reindex(
    index=all_month_years, 
    fill_value=0
)

# Only plot if we have data
if not pivot_month_year_region_complete.empty and len(pivot_month_year_region_complete.columns) > 0:
    
    # Reorder columns to show Innerhalb first, then Ausserhalb (for visual display)
    desired_column_order = [
        'Innerhalb Perimeter, innerhalb Bewässerungsflächen',
        'Innerhalb Perimeter, ausserhalb Bewässerungsflächen'
    ]
    
    # Reorder columns if they exist
    existing_columns = [col for col in desired_column_order if col in pivot_month_year_region_complete.columns]
    pivot_month_year_region_reordered = pivot_month_year_region_complete[existing_columns]
    
    # Create the stacked bar plot with colors matching the new order
    # First column (Innerhalb) gets green, second column (Ausserhalb) gets orange
    pivot_month_year_region_reordered.plot(kind='bar', stacked=True, ax=ax, 
                                          color=['#2ca02c', '#ff7f0e'])
    
    # Customize the plot with German labels
    ax.set_title('ET Blue Bewässerungsvolumen: Innerhalb und Ausserhalb Bewässerungsflächen\n(Perimeter Bibertal, monatliche Werte 2018-2024)', 
                 fontsize=16, fontweight='bold', pad=20)
    ax.set_xlabel('Jahr-Monat', fontsize=20, fontweight='bold')
    ax.set_ylabel('Volumen (m³)', fontsize=20, fontweight='bold')
    
    # Format y-axis with German/Swiss formatting
    ax.yaxis.set_major_formatter(plt.FuncFormatter(format_thousands_german))
    
    # Rotate x-axis labels for better readability and set tick font sizes
    ax.tick_params(axis='x', rotation=45, labelsize=18)
    ax.tick_params(axis='y', labelsize=18)
    
    # Customize legend - order matches the reordered columns
    # First Innerhalb (green), then Ausserhalb (orange)
    legend_labels = ['Innerhalb Bewässerungsflächen', 'Ausserhalb Bewässerungsflächen']
    ax.legend(legend_labels, title=None, loc='upper center', framealpha=0.9, fontsize=20)
    
    # Add grid for better readability
    ax.grid(True, alpha=0.3, axis='y')
else:
    ax.text(0.5, 0.5, 'Keine Daten für Perimeter-Regionen verfügbar', 
            ha='center', va='center', transform=ax.transAxes, fontsize=18)
    ax.set_title('ET Blue Bewässerungsvolumen: Perimeter-Regionen\n(Keine Daten verfügbar)', 
                 fontsize=18, fontweight='bold')

# Adjust layout
plt.tight_layout()
plt.show()

# Print summary statistics for regional month-year analysis - in German
print("\n" + "=" * 80)
print("REGIONALE ZEITREIHENANALYSE")
print("=" * 80)

# Perimeter analysis
perimeter_data = df_regional[df_regional['region_name'].isin(perimeter_regions)]
if not perimeter_data.empty:
    perimeter_monthly = perimeter_data.groupby(['month', 'region_name'])['volume_m3'].sum().unstack(fill_value=0)
    
    print(f"\nMonatliche Volumina in Perimeter-Regionen (m³):")
    # German month names
    month_names_german = {
        '05': 'Mai', '06': 'Juni', '07': 'Juli', '08': 'August', '09': 'September',
        5: 'Mai', 6: 'Juni', 7: 'Juli', 8: 'August', 9: 'September'
    }
    perimeter_monthly.index = [month_names_german.get(month, str(month)) for month in perimeter_monthly.index]
    print(perimeter_monthly.round(0))
else:
    print("Keine Daten für Perimeter-Regionen gefunden.")

# Canton comparison by year
print(f"\n" + "=" * 60)
print("KANTONS-VERGLEICH NACH JAHREN")
print("=" * 60)

canton_regions = ['Kanton Schaffhausen', 'Kanton Zürich', 'Kanton Thurgau']
canton_data = df_regional[df_regional['region_name'].isin(canton_regions)]
if not canton_data.empty:
    canton_yearly = canton_data.groupby(['year', 'region_name'])['volume_m3'].sum().unstack(fill_value=0)
    
    print(f"\nJährliche Volumina nach Kanton (m³):")
    print(canton_yearly.round(0))
else:
    print("Keine Daten für Kantone gefunden.")

# Peak irrigation analysis by region
print(f"\n" + "=" * 60)
print("SPITZENBEWÄSSERUNGS-ANALYSE NACH REGIONEN")
print("=" * 60)

month_names_german = {
    '05': 'Mai', '06': 'Juni', '07': 'Juli', '08': 'August', '09': 'September',
    5: 'Mai', 6: 'Juni', 7: 'Juli', 8: 'August', 9: 'September'
}

for region_name in df_regional['region_name'].unique():
    region_data = df_regional[df_regional['region_name'] == region_name]
    
    # Find peak month-year
    monthly_totals = region_data.groupby(['year', 'month'])['volume_m3'].sum()
    if not monthly_totals.empty and monthly_totals.max() > 0:
        peak_idx = monthly_totals.idxmax()
        peak_volume = monthly_totals.max()
        month_german = month_names_german.get(peak_idx[1], str(peak_idx[1]))
        print(f"{region_name}:")
        print(f"  Spitzenmonat: {month_german} {peak_idx[0]} ({peak_volume:,.0f} m³)")
    else:
        print(f"{region_name}: Keine Bewässerungsdaten")

In [ ]:
##Lets refine the crop classes of Class 1 (vegetable and potato) into more specific categories
vegetables_list = ee.List([
    "Einjährige Freilandgemüse, ohne Konservengemüse",
    "Mehrjährige gärtnerische Freilandkulturen (nicht im Gewächshaus)",
    "Mehrjährige gärtnerische Freilandkulturen",
    "Einjährige Freilandgemüse o. Konservengemüse",
    "Freiland-Konservengemüse", 
    "Freilandgemüse (ohne Kons.gemüse)",
    "Spargel",
    "Ölkürbisse",
    "Oelkürbisse",
    "Rhabarber",
    "Einjährige gärtnerische Freilandkulturen (Blumen, Rollrasen usw.)",
    "Einjä. gärtn. Freilandkult.(Blumen,Rollrasen)",
    "Mehrjährige Gewürz- und Medizinalpflanzen",
    "Soja"#Leguminosen wie Soja sind bei Agroscope als Gemüse klassifiziert
])

kartoffel_list = ee.List([
    "Kartoffeln",
    "Pflanzkartoffeln (Vertragsanbau)"
])

beeren_list = ee.List([
    "Einjährige Beeren (z.B. Erdbeeren)","Einjährige Beeren (Erdbeeren etc.)",
    "Mehrjährige Beeren"
])

others_list = ee.List([
    "Tabak",
])

# Function to reclassify Class 1 into more specific categories
def reclassify_crop(feature):
    crop_name = ee.String(feature.get('nutzung'))
    new_class = feature.get('class')  # Default to original class
    
    # Only reclassify if it's class 1 (vegetables/potatoes)
    new_class = ee.Algorithms.If(
        ee.Number(feature.get('class')).eq(1),
        ee.Algorithms.If(vegetables_list.contains(crop_name), 11,
        ee.Algorithms.If(kartoffel_list.contains(crop_name), 12,
        ee.Algorithms.If(beeren_list.contains(crop_name), 13,
        ee.Algorithms.If(others_list.contains(crop_name), 14, new_class)))),
        ##lets assume second crops (class 5) are also vegetables and reclassify them to 11
        ee.Algorithms.If(ee.Number(feature.get('class')).eq(5), 11, new_class)
    )
    
    return feature.set('refined_class', new_class)


In [ ]:
# Apply reclassification to the full dataset
fc_all_reclassified = fc_all.map(reclassify_crop)

# #print the number of features with refined_class equal to 14
# num_class_14 = fc_all_reclassified.filter(ee.Filter.eq('refined_class', 14)).size().getInfo()
# print('Number of features still in original Class 14:', num_class_14)

# #print the nutzung values of features with refined_class equal to 14
# nutzung_class_14 = fc_all_reclassified.filter(ee.Filter.eq('refined_class', 14)).aggregate_array('nutzung').distinct().getInfo()
# print('Nutzung values of features still in original Class 14:', nutzung_class_14)

In [ ]:
# Add this at the beginning of cells that might produce duplicate output
from IPython.display import clear_output
clear_output(wait=True)

# Calculate annual volumes using admin boundaries instead of canton field
print("\nCalculating annual volumes by admin region, year, and refined class...")

# Apply reclassification to the full dataset
fc_all_reclassified = fc_all.map(reclassify_crop)

# Define regions using admin boundaries
regions = ['Schaffhausen', 'Zuerich', 'Thurgau']

# Get unique years from the data
years_in_data = [2021,2022,2023]#fc_all.aggregate_array('bezugsjahr').distinct().getInfo()
print(f"Years to process: {years_in_data}")

# Calculate annual volumes by admin region, year, and refined class
annual_volumes_data = []

# Calculate summary for each combination
for year in years_in_data:
    print(f"Processing year {year}...")
    
    # Filter by year first
    fc_year = fc_all_reclassified.filter(ee.Filter.eq('bezugsjahr', year))
    
    for region in regions:
        print(f"  Processing region {region}...")
        
        # Filter by region using admin boundaries
        if region == 'Schaffhausen':
            # Canton Schaffhausen
            canton_sh = adm1_units.filter(ee.Filter.eq('NAME', 'Schaffhausen'))
            fc_region = fc_year.filterBounds(canton_sh.geometry())
            
        elif region == 'Zuerich':
            # Canton Zürich
            canton_zh = adm1_units.filter(ee.Filter.eq('NAME', 'Zürich'))
            fc_region = fc_year.filterBounds(canton_zh.geometry())
            
        elif region == 'Thurgau':
            # Canton Thurgau
            canton_tg = adm1_units.filter(ee.Filter.eq('NAME', 'Thurgau'))
            fc_region = fc_year.filterBounds(canton_tg.geometry())
        
        # Get unique refined classes for this region and year
        refined_classes_list = [2,3,4,11,12,13,14]#fc_region.aggregate_array('refined_class').distinct().getInfo()
        
        for refined_class in refined_classes_list:
            # Filter by refined class
            fc_class = fc_region.filter(ee.Filter.eq('refined_class', refined_class))
            
            # Calculate statistics
            total_volume_m3 = fc_class.aggregate_sum('ETblue_vol').getInfo()
            total_area_m2 = fc_class.aggregate_sum('flaeche_m2').getInfo()
            total_area_ha = total_area_m2 / 10000 if total_area_m2 > 0 else 0  # Convert to hectares
            # field_count = fc_class.size().getInfo()
            
            # Calculate average ET blue per hectare
            avg_et_blue_mm = (total_volume_m3 / total_area_ha * 1000) if total_area_ha > 0 else 0  # mm per ha
            
            annual_volumes_data.append({
                'year': year,
                'region': region,
                'refined_class': refined_class,
                'total_volume_m3': total_volume_m3,
                'total_area_ha': total_area_ha,
                # 'field_count': field_count,
                'avg_et_blue_mm': avg_et_blue_mm
            })


In [ ]:

# Convert to DataFrame
import pandas as pd
df_annual_volumes = pd.DataFrame(annual_volumes_data)

# Create class name mapping - keeping your original class definitions
class_mapping = {
        2: 'Mais',
        3: 'Grünland',
        4: 'Zuckerrüben',
        11: 'Gemüse',
        12: 'Kartoffeln', 
        13: 'Beeren',
        14: 'Sonstige'
}

df_annual_volumes['class_name'] = df_annual_volumes['refined_class'].map(class_mapping)

# Sort by year, region, and refined class
df_annual_volumes = df_annual_volumes.sort_values(['year', 'region', 'refined_class'])

# Round numerical columns for better display
df_annual_volumes = df_annual_volumes.round({
    'total_volume_m3': 0,
    'total_area_ha': 0,
    'avg_et_blue_mm': 0
})

# total volume: divide by 0.25 to estimate gross volume assuming 25% efficiency
df_annual_volumes['extracted_volume_m3'] = df_annual_volumes['total_volume_m3'] / 0.25
df_annual_volumes = df_annual_volumes.round({
    'extracted_volume_m3': 0
})

print(f"\nResults DataFrame shape: {df_annual_volumes.shape}")
print("\nFirst 21 rows:")
print(df_annual_volumes.head(21))


In [ ]:
# Create pivot tables for each canton showing class_name (rows) vs years (columns) with extracted volumes
print("\n" + "="*80)
print("PIVOT TABLES BY CANTON - EXTRACTED VOLUMES (m³)")
print("Rows: Class Name | Columns: Years")
print("="*80)

# Get unique regions
unique_regions = df_annual_volumes['region'].unique()

canton_tables = {}

for region in unique_regions:
    print(f"\n{region.upper()} - Extracted Irrigation Volumes (m³)")
    print("-" * 60)
    
    # Filter data for this region
    region_data = df_annual_volumes[df_annual_volumes['region'] == region]
    
    # Create pivot table: class_name as rows, year as columns, extracted_volume_m3 as values
    pivot_table = region_data.pivot_table(
        index='class_name', 
        columns='year', 
        values='extracted_volume_m3', 
        fill_value=0,
        aggfunc='sum'
    )
    
    # Round to nearest integer
    pivot_table = pivot_table.round(0).astype(int)
    
    # Store for later use
    canton_tables[region] = pivot_table
    
    # Display the table
    print(pivot_table)
    
    # # Add row totals (total per class across all years)
    # row_totals = pivot_table.sum(axis=1)
    # print(f"\nTotals by class (across all years):")
    # for class_name, total in row_totals.items():
    #     print(f"  {class_name}: {total:,} m³")
    
    # # Add column totals (total per year across all classes)
    # col_totals = pivot_table.sum(axis=0)
    # print(f"\nTotals by year (across all classes):")
    # for year, total in col_totals.items():
    #     print(f"  {year}: {total:,} m³")
    
    # # Grand total
    # grand_total = pivot_table.sum().sum()
    # print(f"\nGrand total for {region}: {grand_total:,} m³")
    # print("\n")

# Create summary table showing totals by canton and year
print("\n" + "="*80)
print("SUMMARY TABLE - TOTAL EXTRACTED VOLUMES BY CANTON AND YEAR (m³)")
print("="*80)

canton_year_summary = df_annual_volumes.groupby(['region', 'year'])['extracted_volume_m3'].sum().reset_index()
canton_year_pivot = canton_year_summary.pivot(index='region', columns='year', values='extracted_volume_m3').fillna(0).round(0).astype(int)

print(canton_year_pivot)

# Add totals
print(f"\nTotals by canton (across all years):")
canton_totals = canton_year_pivot.sum(axis=1)
for canton, total in canton_totals.items():
    print(f"  {canton}: {total:,} m³")

print(f"\nTotals by year (across all cantons):")
year_totals = canton_year_pivot.sum(axis=0)
for year, total in year_totals.items():
    print(f"  {year}: {total:,} m³")

total_extracted = canton_year_pivot.sum().sum()
print(f"\nTotal extracted volume across all cantons and years: {total_extracted:,} m³")

In [ ]:
# save canton_tables to csv (first append to a single dataframe)
canton_tables_combined = pd.concat(canton_tables, axis=0)
#add a column for canton
canton_tables_combined = canton_tables_combined.reset_index()

canton_tables_combined.to_csv('../data/processed/canton_tables_combined.csv', index=False)



## Load Land Use Data

In [ ]:
# Load land use data
landuse = (
    ee.FeatureCollection('projects/thurgau-irrigation/assets/ZH_SH_TG/Nutzungsflaechen/ZH_SH_TG_2018_Kulturen_with_veg_period_and_crop_class')
    .map(lambda ft: ee.Feature(ft).set('field_id2', ft.id()))
)

print('Land use collection loaded')

print('vegetable area',landuse.filterBounds(perimeter).filter(ee.Filter.eq('class', 1)).aggregate_sum('flaeche_m2').divide(10000).getInfo())
print('vegetable number of fields',landuse.filterBounds(perimeter).filter(ee.Filter.eq('class', 1)).size().getInfo())

In [ ]:
# Add this at the beginning of cells that might produce duplicate output
from IPython.display import clear_output
clear_output(wait=True)


# Load land use data for all years (2018-2024)
landuse_years = {}
landuse_bibertal_years = {}

print("Loading land use data for years 2018-2024...")
print("=" * 60)

for year in years:
    try:
        # Load land use data for this year
        landuse_year = ee.FeatureCollection(
            f'projects/thurgau-irrigation/assets/ZH_SH_TG/Nutzungsflaechen/ZH_SH_TG_{year}_Kulturen_with_veg_period_and_crop_class'
        ).map(lambda ft: ee.Feature(ft).set('field_id2', ft.id()))
        
        # Filter by Bibertal perimeter
        landuse_bibertal = landuse_year.filterBounds(perimeter.geometry())
        
        # Store both full and filtered collections
        landuse_years[year] = landuse_year
        landuse_bibertal_years[year] = landuse_bibertal
        
        # Print basic statistics
        total_fields = landuse_bibertal.size().getInfo()
        total_area = landuse_bibertal.aggregate_sum('flaeche_m2').getInfo()
        
        print(f"{year}: {total_fields} fields, {total_area/10000:.1f} ha total area")
        
        # # Print areas by class for this year
        # classes = {
        #     2: 'Mais',
        #     3: 'Grünland',
        #     4: 'Zuckerrüben',
        #     11: 'Gemüse',
        #     12: 'Kartoffeln', 
        #     13: 'Beeren',
        #     14: 'Sonstige'
        # }
        
        # for class_num, class_name in classes.items():
        #     landuse_class = landuse_bibertal.filter(ee.Filter.eq('class', class_num))
            
        #     # For class 5, also show double cropping vs total
        #     if class_num == 5:
        #         total_class_area = landuse_class.aggregate_sum('flaeche_m2').getInfo()
        #         double_crop_area = landuse_class.filter(ee.Filter.eq('isDoubleCropping', 1)).aggregate_sum('flaeche_m2').getInfo()
        #         total_class_fields = landuse_class.size().getInfo()
        #         double_crop_fields = landuse_class.filter(ee.Filter.eq('isDoubleCropping', 1)).size().getInfo()
                
        #         if total_class_area and total_class_area > 0:
        #             print(f"    Class {class_num} ({class_name}): {total_class_area/10000:.1f} ha ({total_class_fields} fields)")
        #             print(f"        - Double cropping: {double_crop_area/10000:.1f} ha ({double_crop_fields} fields)")
        #     else:
        #         class_area = landuse_class.aggregate_sum('flaeche_m2').getInfo()
        #         class_fields = landuse_class.size().getInfo()
                
        #         if class_area and class_area > 0:
        #             print(f"    Class {class_num} ({class_name}): {class_area/10000:.1f} ha ({class_fields} fields)")
        
    except Exception as e:
        print(f"Error loading {year} land use data: {e}")
        # Create empty collection as fallback
        landuse_years[year] = ee.FeatureCollection([])
        landuse_bibertal_years[year] = ee.FeatureCollection([])

print("\nLand use data loaded for all years")


In [ ]:
region_name = 'Inside_Perimeter_Inside_Bewaesserungsflaechen'
region_data = df_regional[df_regional['region_code'] == region_name]
# print area_m2 per year and class_name
region_summary = region_data.groupby(['year', 'class_name'])['area_m2'].sum()
print(region_summary)

In [ ]:
# Function to reclassify Class 1 into more specific categories
def reclassify_crop_v2(feature):
    crop_name = ee.String(feature.get('nutzung'))
    new_class = feature.get('class')  # Default to original class
    
    # Only reclassify if it's class 1 (vegetables/potatoes)
    new_class = ee.Algorithms.If(
        ee.Number(feature.get('class')).eq(1),
        ee.Algorithms.If(vegetables_list.contains(crop_name), 11,
        ee.Algorithms.If(kartoffel_list.contains(crop_name), 12,
        ee.Algorithms.If(beeren_list.contains(crop_name), 13,
        ee.Algorithms.If(others_list.contains(crop_name), 14, new_class)))),
        ee.Algorithms.If(ee.Number(feature.get('class')).eq(5), 5, new_class)
    )
    
    return feature.set('refined_class', new_class)

In [ ]:
# Add this at the beginning of cells that might produce duplicate output
from IPython.display import clear_output
clear_output(wait=True)

# Dictionary to store feature collections for each year
yearly_collections_v2 = {}

# Base path for the assets
base_path = 'projects/thurgau-irrigation/assets/ZH_SH_TG/ET_blue_per_field_annual_v2_withETA/ETblue_field_stats_'
suffix = '_ETaETc70_ETblue10'

for year in years:
    # Load feature collections for each region
    fc_sh = ee.FeatureCollection(f'{base_path}Schaffhausen_{year}{suffix}')
    fc_zh = ee.FeatureCollection(f'{base_path}Zuerich_{year}{suffix}')
    fc_tg = ee.FeatureCollection(f'{base_path}Thurgau_{year}{suffix}')
    
    # Merge all regions for this year
    fc_year = fc_zh.merge(fc_tg).merge(fc_sh)
    yearly_collections_v2[year] = fc_year.map(reclassify_crop_v2)

# Calculate fraction of irrigated area by year, crop class, and administrative region
def calculate_regional_irrigation_fractions():
    """
    Calculate the fraction of irrigated area for each crop class by year and administrative region
    Using yearly_collections for irrigated areas and land use data for total areas
    
    Special handling for class 5 (second crops): Only considers fields with isDoubleCropping = 1
    """
    
    # Classes mapping
    classes = {
        2: 'Mais',
        3: 'Grünland',
        4: 'Zuckerrüben',
        5: 'Zweite Kulturen',
        11: 'Gemüse',
        12: 'Kartoffeln', 
        13: 'Beeren',
        14: 'Sonstige'
    }
    
    # Administrative regions
    regions = {
        # 'Inside_Perimeter_Outside_Bewaesserungsflaechen': 'Innerhalb Perimeter, ausserhalb Bewässerungsflächen',
        'Inside_Perimeter_Inside_Bewaesserungsflaechen': 'Innerhalb Perimeter, innerhalb Bewässerungsflächen',
        'Canton_Schaffhausen': 'Kanton Schaffhausen'
    }
    
    fraction_results = []
    
    print("Calculating irrigation fractions by year, crop class, and administrative region...")
    print("Using yearly_collections for irrigated areas and land use data for total areas")
    print("=" * 80)
    
    # years = [2022, 2023]
    for year in years:
        print(f"\nProcessing year {year}:")
        
        for region_code, region_name in regions.items():
            print(f"  Processing {region_name}...")
            
            # Get land use data for this year and region
            landuse_year = landuse_years[year].map(reclassify_crop_v2)
            
            # Filter land use by region
            if region_code == 'Inside_Perimeter_Outside_Bewaesserungsflaechen':
                landuse_region = (landuse_year
                                 .filterBounds(perimeter.geometry())
                                 .filter(ee.Filter.Not(ee.Filter.bounds(bewaesserungsflaechen.geometry()))))
            elif region_code == 'Inside_Perimeter_Inside_Bewaesserungsflaechen':
                landuse_region = (landuse_year
                                 .filterBounds(perimeter.geometry())
                                 .filterBounds(bewaesserungsflaechen.geometry()))
            elif region_code == 'Canton_Schaffhausen':
                canton_sh = adm1_units.filter(ee.Filter.eq('NAME', 'Schaffhausen'))
                landuse_region = landuse_year.filterBounds(canton_sh.geometry())
            # elif region_code == 'Canton_Zuerich':
            #     canton_zh = adm1_units.filter(ee.Filter.eq('NAME', 'Zürich'))
            #     landuse_region = landuse_year.filterBounds(canton_zh.geometry())
            # elif region_code == 'Canton_Thurgau':
            #     canton_tg = adm1_units.filter(ee.Filter.eq('NAME', 'Thurgau'))
            #     landuse_region = landuse_year.filterBounds(canton_tg.geometry())
            
            # Get feature collection for this year and region
            fc_year = yearly_collections_v2[year]
            # region_data = df_regional[df_regional['region_code'] == region_code]
            # year_data = region_data[region_data['year'] == year]

            if region_code == 'Inside_Perimeter_Outside_Bewaesserungsflaechen':
                fc_region = (fc_year
                            .filterBounds(perimeter.geometry())
                            .filter(ee.Filter.Not(ee.Filter.bounds(bewaesserungsflaechen.geometry()))))
            elif region_code == 'Inside_Perimeter_Inside_Bewaesserungsflaechen':
                fc_region = (fc_year
                            .filterBounds(perimeter.geometry())
                            .filterBounds(bewaesserungsflaechen.geometry()))
            elif region_code == 'Canton_Schaffhausen':
                canton_sh = adm1_units.filter(ee.Filter.eq('NAME', 'Schaffhausen'))
                fc_region = fc_year.filterBounds(canton_sh.geometry())
            # elif region_code == 'Canton_Zuerich':
            #     canton_zh = adm1_units.filter(ee.Filter.eq('NAME', 'Zürich'))
            #     fc_region = fc_year.filterBounds(canton_zh.geometry())
            # elif region_code == 'Canton_Thurgau':
            #     canton_tg = adm1_units.filter(ee.Filter.eq('NAME', 'Thurgau'))
            #     fc_region = fc_year.filterBounds(canton_tg.geometry())
            
            for class_num, class_name in classes.items():
                try:
                    # Get total area for this crop class from land use data
                    landuse_class = landuse_region.filter(ee.Filter.eq('refined_class', class_num))
                    
                    # # For class 5 (second crops), only consider double cropping fields
                    # if class_num == 5:
                    #     landuse_class = landuse_class.filter(ee.Filter.eq('isDoubleCropping', 1))
                    
                    total_area_m2 = landuse_class.aggregate_sum('flaeche_m2').getInfo()
                    total_area_ha = total_area_m2 / 10000 if total_area_m2 else 0
                    total_fields = landuse_class.size().getInfo()
                    
                    # Get irrigated area for this year, region and class
                    fc_irrigated_class = fc_region.filter(ee.Filter.eq('refined_class', class_num)).filter(ee.Filter.gt('ETblue_mm', 0))
                    
                    irrigated_area_m2 = fc_irrigated_class.aggregate_sum('flaeche_m2').getInfo()
                    irrigated_area_ha = irrigated_area_m2 / 10000 if irrigated_area_m2 else 0
                    # irrigated_fields = fc_irrigated_class.size().getInfo()

                    # irrigated_area_m2 = year_data[year_data['class'] == class_num]['area_m2'].sum()
                    # irrigated_area_ha = irrigated_area_m2 / 10000 if irrigated_area_m2 else 0
                    
                    # Calculate fraction
                    fraction = (irrigated_area_ha / total_area_ha) if total_area_ha > 0 else 0
                    
                    fraction_results.append({
                        'year': year,
                        'class': class_num,
                        'class_name': class_name,
                        'region_code': region_code,
                        'region_name': region_name,
                        'total_area_ha': total_area_ha,
                        'irrigated_area_ha': irrigated_area_ha,
                        'irrigation_fraction': fraction,
                        'total_fields': total_fields,
                        # 'irrigated_fields': irrigated_fields
                    })
                    
                    # if total_area_ha > 0 and region_code == 'Inside_Perimeter_Outside_Bewaesserungsflaechen':
                        # Only print details for first region to avoid spam
                    print(f"  Class {class_num} ({class_name}): {fraction:.1%} " f"({irrigated_area_ha:.1f}/{total_area_ha:.1f} ha)")
                    
                except Exception as e:
                    print(f"    Error processing Class {class_num} ({class_name}) in {region_name}: {e}")
                    fraction_results.append({
                        'year': year,
                        'class': class_num,
                        'class_name': class_name,
                        'region_code': region_code,
                        'region_name': region_name,
                        'total_area_ha': 0,
                        'irrigated_area_ha': 0,
                        'irrigation_fraction': 0,
                        'total_fields': 0,
                        # 'irrigated_fields': 0
                    })
    
    return fraction_results

# Execute the calculation
fraction_results = calculate_regional_irrigation_fractions()

# Convert to DataFrame
df_fractions = pd.DataFrame(fraction_results)

# Save regional data to CSV
output_csv_fractions = '../data/processed/irrigation_fractions_by_region_v3.csv'
df_fractions.to_csv(output_csv_fractions, index=False)

In [ ]:
# Load the CSV to check
output_csv_fractions = '../data/processed/irrigation_fractions_by_region_v3.csv'
df_fractions = pd.read_csv(output_csv_fractions)

# Plot irrigation fractions for region 'Inside_Perimeter_Inside_Bewaesserungsflaechen' over the years for each class_name
import matplotlib.pyplot as plt

# Filter data for the specific region
region_data = df_fractions[df_fractions['region_code'] == 'Inside_Perimeter_Inside_Bewaesserungsflaechen']

# Create pivot table for barplot
pivot_data = region_data.pivot(index='year', columns='class_name', values='irrigation_fraction')

# Create barplot
fig, ax = plt.subplots(figsize=(20, 9))

# Plot stacked or grouped bars
pivot_data.plot(kind='bar', ax=ax, width=0.8)

ax.set_title('Bewässerungsanteil nach Kulturklasse im Perimeterbereich\n(Innerhalb Bewässerungsflächen)', 
             fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Jahr', fontsize=20, fontweight='bold')
ax.set_ylabel('Bewässerungsanteil', fontsize=20, fontweight='bold')
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))
ax.tick_params(axis='x', labelsize=18, rotation=45)
ax.tick_params(axis='y', labelsize=18)
ax.legend(title='Kulturklasse', loc='upper left', framealpha=0.9, fontsize=16)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

#plot the average irrigation fraction across all years for each class_name
import matplotlib.pyplot as plt
# Filter data for the specific region
region_data = df_fractions[df_fractions['region_code'] == 'Inside_Perimeter_Inside_Bewaesserungsflaechen']
# Calculate average irrigation fraction across all years for each class_name
avg_fractions = region_data.groupby('class_name')['irrigation_fraction'].mean()
# Create barplot
fig, ax = plt.subplots(figsize=(12, 6))
avg_fractions.plot(kind='bar', ax=ax, color='#1f77b4', width=0.6)
ax.set_title('Durchschnittlicher Bewässerungsanteil nach Kulturklasse\n(Innerhalb Bewässerungsflächen)', 
             fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Kulturklasse', fontsize=14, fontweight='bold')
ax.set_ylabel('Durchschnittlicher Bewässerungsanteil', fontsize=14, fontweight='bold')
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))
ax.tick_params(axis='x', labelsize=12, rotation=45)
ax.tick_params(axis='y', labelsize=12)
ax.grid(True, alpha=0.3, axis='y')    

#print the average irrigation fraction values
print("Durchschnittlicher Bewässerungsanteil nach Kulturklasse (Innerhalb Bewässerungsflächen):")
for class_name, fraction in avg_fractions.items():
    print(f"  {class_name}: {fraction:.1%}")    

region_data = df_fractions[df_fractions['region_code'] == 'Canton_Schaffhausen']
# Calculate average irrigation fraction across all years for each class_name
avg_fractions = region_data.groupby('class_name')['irrigation_fraction'].mean()
#print the average irrigation fraction values
print("\nDurchschnittlicher Bewässerungsanteil nach Kulturklasse (Kanton Schaffhausen):")
for class_name, fraction in avg_fractions.items():
    print(f"  {class_name}: {fraction:.1%}")
    

In [ ]:
# Add this at the beginning of cells that might produce duplicate output
from IPython.display import clear_output
clear_output(wait=True)


# Calculate fraction of irrigated area by year, crop class, and administrative region
def calculate_regional_irrigation_fractions():
    """
    Calculate the fraction of irrigated area for each crop class by year and administrative region
    Using yearly_collections for irrigated areas and land use data for total areas
    
    Special handling for class 5 (second crops): Only considers fields with isDoubleCropping = 1
    """
    
    # Classes mapping
    classes = {
        1: 'Gemüse und Kartoffeln',
        2: 'Mais',
        3: 'Kunstwiesen',
        4: 'Zuckerrüben',
        5: 'Zweitkulturen'
    }
    
    # Administrative regions
    regions = {
        'Inside_Perimeter_Outside_Bewaesserungsflaechen': 'Innerhalb Perimeter, ausserhalb Bewässerungsflächen',
        'Inside_Perimeter_Inside_Bewaesserungsflaechen': 'Innerhalb Perimeter, innerhalb Bewässerungsflächen',
        'Canton_Schaffhausen': 'Kanton Schaffhausen',
        'Canton_Zuerich': 'Kanton Zürich',
        'Canton_Thurgau': 'Kanton Thurgau'
    }
    
    fraction_results = []
    
    print("Calculating irrigation fractions by year, crop class, and administrative region...")
    print("Using yearly_collections for irrigated areas and land use data for total areas")
    print("=" * 80)
    
    for year in years:
        print(f"\nProcessing year {year}:")
        
        for region_code, region_name in regions.items():
            print(f"  Processing {region_name}...")
            
            # Get land use data for this year and region
            landuse_year = landuse_years[year]
            
            # Filter land use by region
            if region_code == 'Inside_Perimeter_Outside_Bewaesserungsflaechen':
                landuse_region = (landuse_year
                                 .filterBounds(perimeter.geometry())
                                 .filter(ee.Filter.Not(ee.Filter.bounds(bewaesserungsflaechen.geometry()))))
            elif region_code == 'Inside_Perimeter_Inside_Bewaesserungsflaechen':
                landuse_region = (landuse_year
                                 .filterBounds(perimeter.geometry())
                                 .filterBounds(bewaesserungsflaechen.geometry()))
            elif region_code == 'Canton_Schaffhausen':
                canton_sh = adm1_units.filter(ee.Filter.eq('NAME', 'Schaffhausen'))
                landuse_region = landuse_year.filterBounds(canton_sh.geometry())
            elif region_code == 'Canton_Zuerich':
                canton_zh = adm1_units.filter(ee.Filter.eq('NAME', 'Zürich'))
                landuse_region = landuse_year.filterBounds(canton_zh.geometry())
            elif region_code == 'Canton_Thurgau':
                canton_tg = adm1_units.filter(ee.Filter.eq('NAME', 'Thurgau'))
                landuse_region = landuse_year.filterBounds(canton_tg.geometry())
            
            # Get feature collection for this year and region
            fc_year = yearly_collections[year]
            if region_code == 'Inside_Perimeter_Outside_Bewaesserungsflaechen':
                fc_region = (fc_year
                            .filterBounds(perimeter.geometry())
                            .filter(ee.Filter.Not(ee.Filter.bounds(bewaesserungsflaechen.geometry()))))
            elif region_code == 'Inside_Perimeter_Inside_Bewaesserungsflaechen':
                fc_region = (fc_year
                            .filterBounds(perimeter.geometry())
                            .filterBounds(bewaesserungsflaechen.geometry()))
            elif region_code == 'Canton_Schaffhausen':
                canton_sh = adm1_units.filter(ee.Filter.eq('NAME', 'Schaffhausen'))
                fc_region = fc_year.filterBounds(canton_sh.geometry())
            elif region_code == 'Canton_Zuerich':
                canton_zh = adm1_units.filter(ee.Filter.eq('NAME', 'Zürich'))
                fc_region = fc_year.filterBounds(canton_zh.geometry())
            elif region_code == 'Canton_Thurgau':
                canton_tg = adm1_units.filter(ee.Filter.eq('NAME', 'Thurgau'))
                fc_region = fc_year.filterBounds(canton_tg.geometry())
            
            for class_num, class_name in classes.items():
                try:
                    # Get total area for this crop class from land use data
                    landuse_class = landuse_region.filter(ee.Filter.eq('class', class_num))
                    
                    # For class 5 (second crops), only consider double cropping fields
                    if class_num == 5:
                        landuse_class = landuse_class.filter(ee.Filter.eq('isDoubleCropping', 1))
                    
                    total_area_m2 = landuse_class.aggregate_sum('flaeche_m2').getInfo()
                    total_area_ha = total_area_m2 / 10000 if total_area_m2 else 0
                    total_fields = landuse_class.size().getInfo()
                    
                    # Get irrigated area for this year, region and class
                    fc_irrigated_class = fc_region.filter(ee.Filter.eq('class', class_num)).filter(ee.Filter.gt('ETblue_mm', 0))
                    
                    irrigated_area_m2 = fc_irrigated_class.aggregate_sum('flaeche_m2').getInfo()
                    irrigated_area_ha = irrigated_area_m2 / 10000 if irrigated_area_m2 else 0
                    irrigated_fields = fc_irrigated_class.size().getInfo()
                    
                    # Calculate fraction
                    fraction = (irrigated_area_ha / total_area_ha) if total_area_ha > 0 else 0
                    
                    fraction_results.append({
                        'year': year,
                        'class': class_num,
                        'class_name': class_name,
                        'region_code': region_code,
                        'region_name': region_name,
                        'total_area_ha': total_area_ha,
                        'irrigated_area_ha': irrigated_area_ha,
                        'irrigation_fraction': fraction,
                        'total_fields': total_fields,
                        'irrigated_fields': irrigated_fields
                    })
                    
                    # if total_area_ha > 0 and region_code == 'Inside_Perimeter_Outside_Bewaesserungsflaechen':
                        # Only print details for first region to avoid spam
                    print(f"  Class {class_num} ({class_name}): {fraction:.1%} " f"({irrigated_area_ha:.1f}/{total_area_ha:.1f} ha)")
                    
                except Exception as e:
                    print(f"    Error processing Class {class_num} ({class_name}) in {region_name}: {e}")
                    fraction_results.append({
                        'year': year,
                        'class': class_num,
                        'class_name': class_name,
                        'region_code': region_code,
                        'region_name': region_name,
                        'total_area_ha': 0,
                        'irrigated_area_ha': 0,
                        'irrigation_fraction': 0,
                        'total_fields': 0,
                        'irrigated_fields': 0
                    })
    
    return fraction_results

# Execute the calculation
fraction_results = calculate_regional_irrigation_fractions()

# Convert to DataFrame
df_fractions = pd.DataFrame(fraction_results)

# Save regional data to CSV
output_csv_fractions = '../data/processed/irrigation_fractions_by_region_v2.csv'
df_fractions.to_csv(output_csv_fractions, index=False)

In [ ]:
# Load the CSV to check
output_csv_fractions = '../data/processed/irrigation_fractions_by_region_v2.csv'
df_fractions = pd.read_csv(output_csv_fractions)


In [ ]:
# Analyze regional irrigation fractions
print("\n" + "=" * 80)
print("REGIONAL IRRIGATION FRACTION ANALYSIS")
print("=" * 80)

# Summary statistics by region
for region_name in df_fractions['region_name'].unique():
    print(f"\n{region_name}:")
    print("=" * 60)
    
    region_data = df_fractions[df_fractions['region_name'] == region_name]
    
    # Average fractions by crop class for this region
    avg_fractions_region = region_data.groupby(['class', 'class_name'])[['total_area_ha', 'irrigated_area_ha']].sum()
    avg_fractions_region['irrigation_fraction'] = avg_fractions_region['irrigated_area_ha'] / avg_fractions_region['total_area_ha']
    avg_fractions_region = avg_fractions_region.sort_values('class')
    
    print(f"\nOverall irrigation fractions by crop class (2018-2024 average):")
    for (class_num, class_name), row in avg_fractions_region.iterrows():
        if row['total_area_ha'] > 0:
            print(f"  Class {class_num} ({class_name}): {row['irrigation_fraction']:.1%} "
                  f"({row['irrigated_area_ha']:.1f}/{row['total_area_ha']:.1f} ha)")
    
    # Yearly fractions for this region
    pivot_fractions_region = region_data.pivot_table(
        values='irrigation_fraction', 
        index='year', 
        columns='class_name', 
        aggfunc='mean', 
        fill_value=0
    )
    
    if not pivot_fractions_region.empty:
        class_order = ['vegetable and potato', 'maize', 'kunstwiesen', 'zuckerrueben', 'second crops']
        existing_classes = [cls for cls in class_order if cls in pivot_fractions_region.columns]
        if existing_classes:
            pivot_fractions_region_ordered = pivot_fractions_region.reindex(columns=existing_classes, fill_value=0)
            print(f"\nIrrigation fractions by year and crop class:")
            print(pivot_fractions_region_ordered.round(3))

# Comparison across regions
print("\n" + "=" * 80)
print("CROSS-REGIONAL COMPARISON OF IRRIGATION FRACTIONS")
print("=" * 80)

# class_order = ['vegetable and potato', 'maize', 'kunstwiesen', 'zuckerrueben', 'second crops']
# class_order = ['Gemüse', 'Kartoffeln', 'Mais', 'Grünland', 'Zuckerrüben', 'Sonstige']
class_order = ['Gemüse und Kartoffeln', 'Mais', 'Kunstwiesen', 'Zuckerrüben', 'Zweitkulturen']

for class_name in class_order:
    print(f"\n{class_name.upper()}:")
    print("-" * 40)
    
    class_data = df_fractions[df_fractions['class_name'] == class_name]
    
    if not class_data.empty:
        # Calculate average fraction by region for this crop class
        regional_fractions = class_data.groupby('region_name')[['total_area_ha', 'irrigated_area_ha']].sum()
        regional_fractions['irrigation_fraction'] = regional_fractions['irrigated_area_ha'] / regional_fractions['total_area_ha']
        regional_fractions = regional_fractions.sort_values('irrigation_fraction', ascending=False)
        
        for region_name, row in regional_fractions.iterrows():
            if row['total_area_ha'] > 0:
                print(f"  {region_name}: {row['irrigation_fraction']:.1%} "
                      f"({row['irrigated_area_ha']:.1f}/{row['total_area_ha']:.1f} ha)")

# Total agricultural areas by region
print("\n" + "=" * 80)
print("TOTAL AGRICULTURAL AREAS BY REGION")
print("=" * 80)

total_areas_by_region = df_fractions.groupby('region_name')[['total_area_ha', 'irrigated_area_ha']].sum()
total_areas_by_region['overall_fraction'] = total_areas_by_region['irrigated_area_ha'] / total_areas_by_region['total_area_ha']
total_areas_by_region = total_areas_by_region.sort_values('total_area_ha', ascending=False)

print("\nTotal agricultural and irrigated areas by region:")
for region_name, row in total_areas_by_region.iterrows():
    print(f"{region_name}:")
    print(f"  Total agricultural area: {row['total_area_ha']:,.1f} ha")
    print(f"  Total irrigated area: {row['irrigated_area_ha']:,.1f} ha") 
    print(f"  Overall irrigation fraction: {row['overall_fraction']:.1%}")
    print()

### Fraction of Irrigated Area, Bibertal

In [ ]:
# PLOT: Irrigation Fractions and Total Areas Inside Bewässerungsflächen by Year and Crop Class

# Set much larger font sizes for this plot - publication ready
plt.rcParams.update({
    'font.size': 16,           # Base font size increased
    'axes.titlesize': 20,      # Large titles
    'axes.labelsize': 18,      # Larger axis labels
    'xtick.labelsize': 16,     # Larger tick labels
    'ytick.labelsize': 16,     # Larger tick labels
    'legend.fontsize': 14,     # Large legend text
    'legend.title_fontsize': 16 # Large legend title
})

# Focus only on Inside Bewässerungsflächen
region_name = 'Innerhalb Perimeter, innerhalb Bewässerungsflächen'
region_data = df_fractions[df_fractions['region_name'] == region_name]

# Create two subplots stacked vertically (2 rows, 1 column)
fig, axes = plt.subplots(2, 1, figsize=(20, 16))
# class_order = ['vegetable and potato', 'maize', 'kunstwiesen', 'zuckerrueben', 'second crops']
class_order = ['Gemüse und Kartoffeln', 'Mais', 'Kunstwiesen', 'Zuckerrüben', 'Zweitkulturen']

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

# SUBPLOT 1: Irrigation Fractions (top subplot)
ax1 = axes[0]

# Create pivot table for irrigation fractions
pivot_fractions_region = region_data.pivot_table(
    values='irrigation_fraction', 
    index='year', 
    columns='class_name', 
    aggfunc='mean', 
    fill_value=0
)

# Reorder columns if they exist
existing_classes = [cls for cls in class_order if cls in pivot_fractions_region.columns]
if existing_classes:
    pivot_fractions_region_ordered = pivot_fractions_region.reindex(columns=existing_classes, fill_value=0)
    
    # Use only the colors for existing classes
    class_colors = [colors[class_order.index(cls)] for cls in existing_classes]
    
    pivot_fractions_region_ordered.plot(kind='bar', ax=ax1, color=class_colors, width=0.75)
    
    # Customize the first plot
    ax1.set_title('Bewässerungsanteile nach Jahr und Kulturart\nInnerhalb Bewässerungsflächen (2018-2024)', 
                 fontsize=20, fontweight='bold', pad=20)
    ax1.set_xlabel('Jahr', fontsize=18, fontweight='bold')
    ax1.set_ylabel('Bewässerungsanteil', fontsize=18, fontweight='bold')
    ax1.set_ylim(0, 1)
    ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:.0%}'))
    ax1.tick_params(axis='x', rotation=45, labelsize=16)
    ax1.tick_params(axis='y', labelsize=16)
    
    # Add legend inside the first plot
    legend1 = ax1.legend(title=None, loc='upper right', fontsize=20, title_fontsize=16,
                        framealpha=0.95, fancybox=True, shadow=True, 
                        borderpad=1, handlelength=2, handletextpad=0.8)
    legend1.get_frame().set_linewidth(1.5)
    legend1.get_frame().set_edgecolor('gray')
    
    ax1.grid(True, alpha=0.3, axis='y', linewidth=1)

# SUBPLOT 2: Average Total vs Irrigated Areas by Crop Class (bottom subplot)
ax2 = axes[1]

# Calculate average areas by crop class (2018-2024 combined)
class_totals = region_data.groupby('class_name')[['total_area_ha', 'irrigated_area_ha']].mean()

# Reorder according to class_order and filter existing classes
if existing_classes:
    class_totals_ordered = class_totals.reindex(existing_classes, fill_value=0)
    
    # Prepare data for grouped bar plot
    crop_classes = class_totals_ordered.index
    x_pos = np.arange(len(crop_classes))
    width = 0.35
    
    # Create grouped bars
    bars1 = ax2.bar(x_pos - width/2, class_totals_ordered['total_area_ha'], width, 
                    label='Gesamte Landwirtschaftsfläche', color='lightblue', alpha=0.8)
    bars2 = ax2.bar(x_pos + width/2, class_totals_ordered['irrigated_area_ha'], width, 
                    label='Gesamte bewässerte Fläche', color='darkblue', alpha=0.8)
    
    # Customize the second plot
    ax2.set_title('Durchschnittliche Gesamt- vs. bewässerte Flächen nach Kulturart\nGebiet Bibertal (2018-2024)', 
                 fontsize=20, fontweight='bold', pad=20)
    ax2.set_xlabel('Kulturart', fontsize=18, fontweight='bold')
    ax2.set_ylabel('Fläche (ha)', fontsize=18, fontweight='bold')
    ax2.set_xticks(x_pos)
    ax2.set_xticklabels(crop_classes, rotation=45, ha='right')
    ax2.tick_params(axis='x', labelsize=14)
    ax2.tick_params(axis='y', labelsize=16)
    
    # Add fraction labels only on dark blue bars (irrigated area)
    for i, bar2 in enumerate(bars2):
        total_val = class_totals_ordered.iloc[i]['total_area_ha']
        irrigated_val = class_totals_ordered.iloc[i]['irrigated_area_ha']
        
        if irrigated_val > 0 and total_val > 0:
            fraction = irrigated_val / total_val
            ax2.text(bar2.get_x() + bar2.get_width()/2, bar2.get_height() + 5,
                     f'{fraction:.1%}', ha='center', va='bottom', fontsize=12, fontweight='bold')
    
    # Add legend at center top
    legend2 = ax2.legend(title=None, loc='upper center', fontsize=20, title_fontsize=16,
                        framealpha=0.95, fancybox=True, shadow=True, 
                        borderpad=1, handlelength=2, handletextpad=0.8)
    legend2.get_frame().set_linewidth(1.5)
    legend2.get_frame().set_edgecolor('gray')
    
    ax2.grid(True, alpha=0.3, axis='y', linewidth=1)

# Add main title
# plt.suptitle('Landwirtschaftliche Bewässerungsanalyse: Innerhalb Bewässerungsflächen (2018-2024)', 
#              fontsize=22, fontweight='bold', y=0.98)

plt.tight_layout()
plt.show()

# Reset font parameters to default after plotting
plt.rcParams.update({
    'font.size': 10,
    'axes.titlesize': 'large',
    'axes.labelsize': 'medium',
    'xtick.labelsize': 'medium',
    'ytick.labelsize': 'medium',
    'legend.fontsize': 'medium',
    'legend.title_fontsize': None
})

# Print summary for Inside Bewässerungsflächen
print("\n" + "=" * 70)
print("BEWÄSSERUNGSANALYSE: INNERHALB BEWÄSSERUNGSFLÄCHEN")
print("=" * 70)

if not region_data.empty:
    # Summary by year
    yearly_summary = region_data.groupby('year')[['total_area_ha', 'irrigated_area_ha']].sum()
    yearly_summary['irrigation_fraction'] = yearly_summary['irrigated_area_ha'] / yearly_summary['total_area_ha']
    
    print(f"\nGesamter Bewässerungsanteil nach Jahr:")
    for year, row in yearly_summary.iterrows():
        print(f"  {year}: {row['irrigation_fraction']:.1%} ({row['irrigated_area_ha']:.1f}/{row['total_area_ha']:.1f} ha)")
    
    # Summary by crop class (all years combined)
    class_summary = region_data.groupby('class_name')[['total_area_ha', 'irrigated_area_ha']].mean()
    class_summary['irrigation_fraction'] = class_summary['irrigated_area_ha'] / class_summary['total_area_ha']
    class_summary = class_summary.sort_values('irrigation_fraction', ascending=False)
    
    print(f"\nDurchschnittlicher Bewässerungsanteil nach Kulturart (2018-2024):")
    for class_name, row in class_summary.iterrows():
        print(f"  {class_name}: {row['irrigation_fraction']:.1%} ({row['irrigated_area_ha']:.1f}/{row['total_area_ha']:.1f} ha)")
        
    # Overall statistics
    total_area = region_data['total_area_ha'].sum()
    total_irrigated = region_data['irrigated_area_ha'].sum()
    overall_fraction = total_irrigated / total_area if total_area > 0 else 0
    
    print(f"\nGesamtstatistik (2018-2024):")
    print(f"  Gesamte Landwirtschaftsfläche: {total_area:,.1f} ha")
    print(f"  Gesamte bewässerte Fläche: {total_irrigated:,.1f} ha") 
    print(f"  Gesamter Bewässerungsanteil: {overall_fraction:.1%}")
else:
    print("Keine Daten für Innerhalb Bewässerungsflächen verfügbar")

In [ ]:
# Create the two requested plots with larger fonts and internal legends

# Set larger font sizes globally for this plot
plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 16,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'legend.fontsize': 14,
    'legend.title_fontsize': 14
})

# PLOT 1: Overall irrigation fraction comparison (Inside vs ausserhalb BewässerungsflächenSonstige)
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

# Plot 1: Overall irrigation fraction by year
perimeter_regions = [
    'Innerhalb Perimeter, ausserhalb Bewässerungsflächen',
    'Innerhalb Perimeter, innerhalb Bewässerungsflächen'
]

ax1 = axes[0]
colors_perimeter = ['#ff7f0e', '#2ca02c']

# Calculate overall irrigation fractions by year for perimeter regions
overall_fractions_by_year = []
years_list = sorted(df_fractions['year'].unique())

for year in years_list:
    year_data = df_fractions[df_fractions['year'] == year]
    
    for i, region_name in enumerate(perimeter_regions):
        region_year_data = year_data[year_data['region_name'] == region_name]
        
        if not region_year_data.empty:
            total_area = region_year_data['total_area_ha'].sum()
            irrigated_area = region_year_data['irrigated_area_ha'].sum()
            overall_fraction = (irrigated_area / total_area) if total_area > 0 else 0
            
            overall_fractions_by_year.append({
                'year': year,
                'region_name': region_name,
                'fraction': overall_fraction
            })

df_overall_fractions = pd.DataFrame(overall_fractions_by_year)

# Create pivot table for plotting
pivot_overall = df_overall_fractions.pivot_table(
    values='fraction', 
    index='year', 
    columns='region_name', 
    fill_value=0
)

# Reorder columns to match our region list
pivot_overall_ordered = pivot_overall.reindex(columns=perimeter_regions, fill_value=0)

# Plot overall fractions
pivot_overall_ordered.plot(kind='bar', ax=ax1, color=colors_perimeter, width=0.7)

ax1.set_title('Anteil Bewässerungsfläche nach Jahr\n(alle Kulturarten zusammengefasst)', 
              fontsize=18, fontweight='bold', pad=20)
ax1.set_xlabel('Jahr', fontsize=20, fontweight='bold')
ax1.set_ylabel('Anteil Bewässerungsfläche', fontsize=20, fontweight='bold')
ax1.set_ylim(0, max(pivot_overall_ordered.max()) * 1.15)
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:.0%}'))
ax1.tick_params(axis='x', rotation=45, labelsize=20)
ax1.tick_params(axis='y', labelsize=20)

# Move legend inside the plot (upper right)
legend_labels = ['Ausserhalb Bewässerungsflächen', 'Innerhalb Bewässerungsflächen']
ax1.legend(legend_labels,title=None, loc='upper right', fontsize=20, title_fontsize=20,
          framealpha=0.9, fancybox=True, shadow=True)
ax1.grid(True, alpha=0.3, axis='y')

# Plot 2: Combined crop class fractions in a single subplot
ax2 = axes[1]

# Prepare data for combined plot - show all classes for both regions
# class_order = ['vegetable and potato', 'maize', 'kunstwiesen', 'zuckerrueben', 'second crops']
class_order = ['Gemüse und Kartoffeln', 'Mais', 'Kunstwiesen', 'Zuckerrüben', 'Zweitkulturen']

colors_classes = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

# Calculate average fractions across all years for each region and class
avg_fractions_combined = []

for region_name in perimeter_regions:
    region_data = df_fractions[df_fractions['region_name'] == region_name]
    
    for class_name in class_order:
        class_region_data = region_data[region_data['class_name'] == class_name]
        
        if not class_region_data.empty:
            total_area = class_region_data['total_area_ha'].sum()
            irrigated_area = class_region_data['irrigated_area_ha'].sum()
            avg_fraction = (irrigated_area / total_area) if total_area > 0 else 0
            
            avg_fractions_combined.append({
                'region_name': region_name,
                'class_name': class_name,
                'fraction': avg_fraction,
                'total_area': total_area,
                'irrigated_area': irrigated_area
            })

df_avg_combined = pd.DataFrame(avg_fractions_combined)

# Create grouped bar plot
regions_short = ['Ausserhalb Bewässerungsflächen', 'Innerhalb Bewässerungsflächen']
x_pos = np.arange(len(class_order))
width = 0.35

# Get data for each region
outside_fractions = []
inside_fractions = []

for class_name in class_order:
    # Outside Bewässerungsflächen
    outside_data = df_avg_combined[
        (df_avg_combined['region_name'] == 'Innerhalb Perimeter, ausserhalb Bewässerungsflächen') &
        (df_avg_combined['class_name'] == class_name)
    ]
    outside_frac = outside_data['fraction'].values[0] if len(outside_data) > 0 else 0
    outside_fractions.append(outside_frac)
    
    # Inside Bewässerungsflächen
    inside_data = df_avg_combined[
        (df_avg_combined['region_name'] == 'Innerhalb Perimeter, innerhalb Bewässerungsflächen') &
        (df_avg_combined['class_name'] == class_name)
    ]
    inside_frac = inside_data['fraction'].values[0] if len(inside_data) > 0 else 0
    inside_fractions.append(inside_frac)

# Create bars
bars1 = ax2.bar(x_pos - width/2, outside_fractions, width, 
                label='Ausserhalb Bewässerungsflächen', color=colors_perimeter[0], alpha=0.8)
bars2 = ax2.bar(x_pos + width/2, inside_fractions, width, 
                label='Innerhalb Bewässerungsflächen', color=colors_perimeter[1], alpha=0.8)

ax2.set_title('Durchschnittlicher Anteil Bewässerungsfläche nach Kulturart\n(2018-2024 zusammengefasst)', 
              fontsize=18, fontweight='bold', pad=20)
# ax2.set_xlabel('Kulturart', fontsize=20, fontweight='bold')
ax2.set_ylabel('Anteil Bewässerungsfläche', fontsize=20, fontweight='bold')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(class_order, rotation=45, ha='right', fontsize=20)
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:.0%}'))
ax2.tick_params(axis='y', labelsize=20)

# Move legend inside the plot (upper right)
ax2.legend(title=None, loc='upper right', fontsize=20, title_fontsize=20,
          framealpha=0.9, fancybox=True, shadow=True)
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels on bars with larger font
for i, (bar1, bar2) in enumerate(zip(bars1, bars2)):
    if outside_fractions[i] > 0:
        ax2.text(bar1.get_x() + bar1.get_width()/2, bar1.get_height() + 0.01,
                f'{outside_fractions[i]:.1%}', ha='center', va='bottom', fontsize=14, fontweight='bold')
    if inside_fractions[i] > 0:
        ax2.text(bar2.get_x() + bar2.get_width()/2, bar2.get_height() + 0.01,
                f'{inside_fractions[i]:.1%}', ha='center', va='bottom', fontsize=14, fontweight='bold')

# plt.suptitle('Bewässerungsanalyse: Bibertal Perimeter-Regionen\n(2018-2024)', 
#              fontsize=20, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

# # Reset font parameters to default
# plt.rcParams.update({
#     'font.size': 30,
#     'axes.titlesize': 'large',
#     'axes.labelsize': 'large',
#     'xtick.labelsize': 'large',
#     'ytick.labelsize': 'large',
#     'legend.fontsize': 'large',
#     'legend.title_fontsize': None
# })

# Print summary statistics
print("\n" + "=" * 80)
print("ZUSAMMENFASSUNG: BIBERTAL BEWÄSSERUNGSANALYSE")
print("=" * 80)

print(f"\nGesamte Bewässerungsanteile (2018-2024 Durchschnitt):")
for region_name in perimeter_regions:
    region_data = df_fractions[df_fractions['region_name'] == region_name]
    total_area = region_data['total_area_ha'].sum()
    irrigated_area = region_data['irrigated_area_ha'].sum()
    overall_fraction = (irrigated_area / total_area) if total_area > 0 else 0
    
    print(f"  {region_name}:")
    print(f"    Gesamtanteil: {overall_fraction:.1%}")
    print(f"    Gesamtfläche: {total_area:,.1f} ha")
    print(f"    Bewässerte Fläche: {irrigated_area:,.1f} ha")

print(f"\nKulturspezifische Anteile (2018-2024 Durchschnitt):")
for class_name in class_order:
    print(f"\n  {class_name}:")
    for region_name in perimeter_regions:
        class_region_data = df_fractions[
            (df_fractions['region_name'] == region_name) &
            (df_fractions['class_name'] == class_name)
        ]
        
        if not class_region_data.empty:
            total_area = class_region_data['total_area_ha'].sum()
            irrigated_area = class_region_data['irrigated_area_ha'].sum()
            fraction = (irrigated_area / total_area) if total_area > 0 else 0
            
            region_short = 'Ausserhalb' if 'ausserhalb' in region_name else 'Innerhalb'
            print(f"    {region_short} Bewässerungsflächen: {fraction:.1%} ({irrigated_area:.1f}/{total_area:.1f} ha)")

### Anteil der bewässerten Fläche, drei Kantone

In [ ]:
# PLOT 2: Three cantons comparison
canton_regions = ['Kanton Schaffhausen', 'Kanton Zürich', 'Kanton Thurgau']

fig, axes = plt.subplots(1, 3, figsize=(20, 8))

# Create irrigation fraction plot for each canton
for i, region_name in enumerate(canton_regions):
    ax = axes[i]
    region_data = df_fractions[df_fractions['region_name'] == region_name]
    
    # Create pivot table for this region
    pivot_fractions_region = region_data.pivot_table(
        values='irrigation_fraction', 
        index='year', 
        columns='class_name', 
        aggfunc='mean', 
        fill_value=0
    )
    
    # Reorder columns if they exist
    existing_classes = [cls for cls in class_order if cls in pivot_fractions_region.columns]
    if existing_classes:
        pivot_fractions_region_ordered = pivot_fractions_region.reindex(columns=existing_classes, fill_value=0)
        
        # Use only the colors for existing classes
        class_colors = [colors[class_order.index(cls)] for cls in existing_classes]
        
        pivot_fractions_region_ordered.plot(kind='bar', ax=ax, color=class_colors)
        
        # Customize the plot
        ax.set_title(f'{region_name}\nBewässerungsanteile (2018-2024)', fontsize=12, fontweight='bold')
        ax.set_xlabel('Jahr', fontsize=11)
        ax.set_ylabel('Bewässerungsanteil', fontsize=11)
        ax.set_ylim(0, 1)
        ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:.0%}'))
        ax.tick_params(axis='x', rotation=45, labelsize=9)
        ax.tick_params(axis='y', labelsize=9)
        
        # Add legend inside each plot
        if i == 0:
            ax.legend(title='Kulturart', loc='upper right', fontsize=9, title_fontsize=10,
                     framealpha=0.95, fancybox=True, shadow=True)
        else:
            ax.legend().set_visible(False)
            
        ax.grid(True, alpha=0.3, axis='y')
    else:
        ax.text(0.5, 0.5, 'Keine Bewässerungsdaten', ha='center', va='center', transform=ax.transAxes)
        ax.set_title(f'{region_name}\n(Keine Daten)', fontsize=12)

plt.suptitle('Bewässerungsanteile nach Kanton\n(2018-2024)', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# PLOT 3: Average irrigation fractions by crop class (2018-2024 combined) for three cantons

# Calculate average irrigation fractions by crop class for each canton (2018-2024)
canton_class_summary = {}
for region_name in canton_regions:
    region_data = df_fractions[df_fractions['region_name'] == region_name]
    if not region_data.empty:
        # Group by class and calculate weighted average irrigation fraction
        class_summary = region_data.groupby('class_name').agg({
            'total_area_ha': 'sum',
            'irrigated_area_ha': 'sum'
        })
        class_summary['irrigation_fraction'] = class_summary['irrigated_area_ha'] / class_summary['total_area_ha']
        class_summary = class_summary[class_summary['total_area_ha'] > 0]  # Filter out classes with no area
        canton_class_summary[region_name] = class_summary

# Create comparison plot
fig, ax = plt.subplots(figsize=(14, 8))

# Prepare data for grouped bar plot
crop_classes_all = set()
for region_data in canton_class_summary.values():
    crop_classes_all.update(region_data.index)

# Filter to only existing classes in our order
existing_classes = [cls for cls in class_order if cls in crop_classes_all]

if existing_classes:
    # Create data matrix
    canton_data = []
    canton_labels = []
    
    for region_name in canton_regions:
        if region_name in canton_class_summary:
            region_short = region_name.replace('Canton ', '')
            canton_labels.append(region_short)
            
            # Get fractions for each class, fill with 0 if not present
            fractions = []
            for class_name in existing_classes:
                if class_name in canton_class_summary[region_name].index:
                    fraction = canton_class_summary[region_name].loc[class_name, 'irrigation_fraction']
                    fractions.append(fraction)
                else:
                    fractions.append(0)
            canton_data.append(fractions)
    
    # Convert to numpy array for easier handling
    canton_data = np.array(canton_data)
    
    # Set up the plot
    x_pos = np.arange(len(existing_classes))
    width = 0.25  # Width of bars
    
    # Create bars for each canton
    colors_cantons = ['#1f77b4', '#ff7f0e', '#2ca02c']  # Different colors for cantons
    
    for i, (canton_label, canton_fractions) in enumerate(zip(canton_labels, canton_data)):
        offset = (i - 1) * width  # Center the bars
        bars = ax.bar(x_pos + offset, canton_fractions, width, 
                     label=canton_label, color=colors_cantons[i], alpha=0.8)
        
        # Add value labels on bars
        for j, bar in enumerate(bars):
            height = bar.get_height()
            if height > 0:
                ax.text(bar.get_x() + bar.get_width()/2, height + 0.01,
                       f'{height:.1%}', ha='center', va='bottom', 
                       fontsize=14, fontweight='bold')
    
    # Customize the plot
    ax.set_title('Durchschnittlicher Anteil Bewässerungsfläche nach Kulturart\n(2018-2024 zusammengefasst)', 
                fontsize=16, fontweight='bold', pad=20)
    ax.set_xlabel('Kulturart', fontsize=16, fontweight='bold')
    ax.set_ylabel('Anteil Bewässerungsfläche', fontsize=16, fontweight='bold')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(class_order, rotation=45, ha='right', fontsize=16)
    ax.tick_params(axis='y', labelsize=16)


    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:.0%}'))
    
    # Add legend inside the plot
    ax.legend(loc='upper right', fontsize=14, title_fontsize=14,
             framealpha=0.95, fancybox=True, shadow=True,
             borderpad=1, handlelength=2, handletextpad=0.8)
    
    ax.grid(True, alpha=0.3, axis='y', linewidth=1)
    ax.set_ylim(0, max(canton_data.max() * 1.15, 0.1))  # Set reasonable y-limit
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\n" + "=" * 80)
    print("DURCHSCHNITTLICHE BEWÄSSERUNGSANTEILE NACH KULTURART (2018-2024)")
    print("=" * 80)
    
    for region_name in canton_regions:
        if region_name in canton_class_summary:
            print(f"\n{region_name}:")
            class_data = canton_class_summary[region_name].sort_values('irrigation_fraction', ascending=False)
            for class_name, row in class_data.iterrows():
                print(f"  {class_name}: {row['irrigation_fraction']:.1%} "
                      f"({row['irrigated_area_ha']:.1f}/{row['total_area_ha']:.1f} ha)")
else:
    print("Keine Kulturart-Daten für den Vergleich verfügbar")

In [ ]:
# Create comparison of total vs irrigated areas by region
fig, ax = plt.subplots(figsize=(16, 8))

# Calculate totals by region and crop class
regional_areas = df_fractions.groupby(['region_name', 'class_name'])[['total_area_ha', 'irrigated_area_ha']].sum()

# Create data for grouped bar chart
regions = df_fractions['region_name'].unique()
n_regions = len(regions)
n_classes = len(class_order)

# Set up positions for bars
x = np.arange(n_regions)
width = 0.15

# Plot bars for each crop class
for i, class_name in enumerate(class_order):
    total_areas = []
    irrigated_areas = []
    
    for region_name in regions:
        try:
            region_class_data = regional_areas.loc[(region_name, class_name)]
            total_areas.append(region_class_data['total_area_ha'])
            irrigated_areas.append(region_class_data['irrigated_area_ha'])
        except KeyError:
            total_areas.append(0)
            irrigated_areas.append(0)
    
    # Plot total areas (lighter color)
    ax.bar(x + i*width - width*2, total_areas, width, 
           label=f'{class_name} (Total)' if i == 0 else '', 
           color=colors[i], alpha=0.3)
    
    # Plot irrigated areas (darker color)
    ax.bar(x + i*width - width*2, irrigated_areas, width,
           label=f'{class_name} (Irrigated)' if i == 0 else '',
           color=colors[i], alpha=0.8)

ax.set_title('Total vs Irrigated Areas by Region and Crop Class\n(2018-2024)', 
             fontsize=14, fontweight='bold', pad=20)
ax.set_xlabel('Administrative Region', fontsize=12, fontweight='bold')
ax.set_ylabel('Area (ha)', fontsize=12, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(regions, rotation=45, ha='right', fontsize=10)

# Create custom legend
from matplotlib.patches import Patch
legend_elements = []
for i, class_name in enumerate(class_order):
    legend_elements.append(Patch(facecolor=colors[i], alpha=0.3, label=f'{class_name} (Total)'))
    legend_elements.append(Patch(facecolor=colors[i], alpha=0.8, label=f'{class_name} (Irrigated)'))

ax.legend(handles=legend_elements, bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Summary statistics
print("\n" + "=" * 80)
print("SUMMARY OF REGIONAL IRRIGATION FRACTIONS")
print("=" * 80)

# Find region with highest overall irrigation fraction
regional_totals = df_fractions.groupby('region_name')[['total_area_ha', 'irrigated_area_ha']].sum()
regional_totals['overall_fraction'] = regional_totals['irrigated_area_ha'] / regional_totals['total_area_ha']
regional_totals = regional_totals.sort_values('overall_fraction', ascending=False)

print(f"\nRegions ranked by overall irrigation fraction:")
for region_name, row in regional_totals.iterrows():
    if row['total_area_ha'] > 0:
        print(f"  {region_name}: {row['overall_fraction']:.1%}")

print(f"\nPerimeter comparison:")
perimeter_regions = [
    'Innerhalb Perimeter, ausserhalb Bewässerungsflächen',
    'Innerhalb Perimeter, innerhalb Bewässerungsflächen'
]

perimeter_fractions = regional_totals.loc[regional_totals.index.isin(perimeter_regions)]
for region_name, row in perimeter_fractions.iterrows():
    print(f"  {region_name}: {row['overall_fraction']:.1%} "
          f"({row['irrigated_area_ha']:.1f}/{row['total_area_ha']:.1f} ha)")

## ETaETc Analysis

In [ ]:
from collections import defaultdict

# Define years to process (client-side)
years = [2018, 2019, 2020, 2021, 2022, 2023, 2024]

# Client-side loop to load all feature collections and tag with year
all_collections = []
for year in years:
    # Load feature collections for each canton
    fc_sh = ee.FeatureCollection(f'projects/thurgau-irrigation/assets/ZH_SH_TG/ET_blue_per_field_filtered_v2/ETblue_field_ts_Schaffhausen_{year}_ETaETc70_ETblue10')
    fc_zh = ee.FeatureCollection(f'projects/thurgau-irrigation/assets/ZH_SH_TG/ET_blue_per_field_filtered_v2/ETblue_field_ts_Zuerich_{year}_ETaETc70_ETblue10')
    fc_tg = ee.FeatureCollection(f'projects/thurgau-irrigation/assets/ZH_SH_TG/ET_blue_per_field_filtered_v2/ETblue_field_ts_Thurgau_{year}_ETaETc70_ETblue10')
    
    # Merge and add year property to each feature
    def add_year(feature):
        return feature.set('year', year)
    
    yearly_collection = fc_sh.merge(fc_zh).merge(fc_tg).map(add_year)
    
    # Load landuse for this year
    landuse = ee.FeatureCollection(f'projects/thurgau-irrigation/assets/ZH_SH_TG/Nutzungsflaechen/ZH_SH_TG_{year}_Kulturen_with_veg_period_and_crop_class')
    
    # Filter by perimeter (buffer by 5000 m to include nearby fields)
    field_ids = landuse.filterBounds(perimeter.geometry().buffer(5000,100)).aggregate_array('field_id')
    yearly_collection = yearly_collection.filter(ee.Filter.inList('field_id', field_ids))
    
    all_collections.append(yearly_collection)

# Merge all years into one collection
fields4stats = ee.FeatureCollection(all_collections).flatten()

# Get unique year-date combinations
years_server = ee.List([2018, 2019, 2020, 2021, 2022, 2023, 2024])

# Map over years (server-side)
def process_year(year):
    year_data = fields4stats.filter(ee.Filter.eq('year', year))
    dates = year_data.sort('date').aggregate_array('date').distinct()
    
    # Calculate stats for each date
    def process_date(date):
        date_data = year_data.filter(ee.Filter.eq('date', date))
        mean = date_data.aggregate_mean('ETa_ETc')
        std = date_data.aggregate_sample_sd('ETa_ETc')
        return ee.Feature(None, {
            'year': year,
            'date': date,
            'mean': mean,
            'std': std
        })
    
    ETaETc_stats = dates.map(process_date)
    return ETaETc_stats

yearly_stats = years_server.map(process_year).flatten()

#pull to client side and print
yearly_stats = ee.FeatureCollection(yearly_stats)
stats_list = yearly_stats.getInfo()['features']
print(f"\nETa_ETc Statistics by Year and Date ({len(stats_list)} records):")
print("=" * 60)

# Group by year for better organization
stats_by_year = defaultdict(list)

for feature in stats_list:
    props = feature['properties']
    year = props['year']
    date = props['date']
    mean = props['mean']
    std = props['std']
    
    # Skip features with no data (None values)
    if mean is not None:
        stats_by_year[year].append({
            'date': date,
            'mean': mean,
            'std': std if std is not None else 0.0
        })

# Print organized results
for year in sorted(stats_by_year.keys()):
    print(f"\nYear {year} ({len(stats_by_year[year])} dates):")
    print("-" * 40)
    
    # Sort by date within each year
    year_data = sorted(stats_by_year[year], key=lambda x: x['date'])
    
    for record in year_data:
        date = record['date']
        mean = record['mean']
        std = record['std']
        print(f"  {date}: Mean ETa_ETc = {mean:.4f}, Std Dev = {std:.4f}")
    
    # Calculate year summary
    if year_data:
        year_means = [r['mean'] for r in year_data]
        year_avg = sum(year_means) / len(year_means)
        year_min = min(year_means)
        year_max = max(year_means)
        
        print(f"  Year {year} Summary:")
        print(f"    Average: {year_avg:.4f}")
        print(f"    Range: {year_min:.4f} - {year_max:.4f}")

# Overall summary
all_means = []
for year_data in stats_by_year.values():
    all_means.extend([r['mean'] for r in year_data])

if all_means:
    print(f"\nOverall Summary (All Years):")
    print("=" * 30)
    print(f"Total records: {len(all_means)}")
    print(f"Overall average ETa_ETc: {sum(all_means)/len(all_means):.4f}")
    print(f"Overall range: {min(all_means):.4f} - {max(all_means):.4f}")

# Optional: Export to CSV or asset if needed
# task = ee.batch.Export.table.toDrive(
#     collection=ee.FeatureCollection(yearly_stats),
#     description='ETaETc_stats_2018_2024',
#     fileFormat='CSV'
# )
# task.start()

## Plot ETa/ETc Time Series with Error Bars

Create a plot similar to the previous example but using the stats_list data with error bars based on standard deviations.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import numpy as np

def plot_etaetc_with_error_bars(stats_data, title="ETa/ETc Ratio Time Series"):
    """
    Plot ETa/ETc time series with error bars from stats_list data.
    
    Args:
        stats_data: List of features from GEE with properties: year, date, mean, std
        title: Title for the plot
    """
    # Convert stats_list to DataFrame
    data_rows = []
    for feature in stats_data:
        props = feature['properties']
        # Parse the date string to datetime
        date_str = props['date']
        try:
            # Handle different possible date formats
            if len(date_str) == 10:  # YYYY-MM-DD format
                date_obj = datetime.strptime(date_str, '%Y-%m-%d').date()
            else:
                # Try other common formats
                date_obj = datetime.strptime(date_str, '%Y-%m-%d').date()
        except:
            print(f"Could not parse date: {date_str}")
            continue
            
        data_rows.append({
            'date': date_obj,
            'year': props['year'],
            'mean': props['mean'],
            'std': props['std']
        })
    
    df = pd.DataFrame(data_rows)
    
    # Replace NaN standard deviations with 0 (no error bars for those points)
    df['std'] = df['std'].fillna(0)
    
    # Filter out rows with null mean values and remove zero or negative means
    df = df.dropna(subset=['mean'])
    df = df[df['mean'] > 0]  # Remove zero or negative values
    
    # Sort by date
    df = df.sort_values('date')
    
    # Reset styles to avoid unintended global settings
    sns.reset_orig()
    
    # Create the plot
    plt.figure(figsize=(14, 8))
    
    # Plot all points with error bars using single color (blue)
    plt.errorbar(
        df['date'], 
        df['mean'],
        yerr=df['std'],
        fmt='o',
        color='blue',
        capsize=3,
        capthick=1,
        markersize=6,
        alpha=0.8
    )

    plt.gca().tick_params(axis='x', labelsize=16)
    plt.gca().tick_params(axis='y', labelsize=16)
    # Add threshold line (commonly used value of 0.7 for irrigation detection)
    plt.axhline(y=0.7, color='r', linestyle='--', alpha=0.7, linewidth=2)
    
    # Customize the plot with German text
    plt.title(title, fontsize=18, fontweight='bold')
    plt.xlabel('Jahr', fontsize=16)
    plt.ylabel('ETa/ETc Verhältnis', fontsize=16)
    plt.grid(True, linestyle='--', alpha=0.3)
    
    # Set up x-axis to show years with August 1st as tick marks
    years = sorted(df['year'].unique())
    august_dates = [datetime(year, 8, 1).date() for year in years]
    plt.xticks(august_dates, [str(year) for year in years], rotation=0)
    
    # Set y-axis limits for better visualization
    plt.ylim(0, max(df['mean'] + df['std']) * 1.01)
    
    # Remove top and right spines
    sns.despine()
    
    # Tight layout to prevent label cutoff
    plt.tight_layout()
    plt.show()
    
    # Print some summary statistics in German
    print(f"\nZusammenfassung der Statistik:")
    print(f"Gesamtanzahl Datenpunkte: {len(df)}")
    print(f"Datumsbereich: {df['date'].min()} bis {df['date'].max()}")
    print(f"Mittleres ETa/ETc: {df['mean'].mean():.3f} ± {df['mean'].std():.3f}")
    print(f"Abgedeckte Jahre: {', '.join(map(str, sorted(df['year'].unique())))}")

# Create the plot using the stats_list data directly
plot_etaetc_with_error_bars(stats_list, "ETa/ETc Verhältnis - Bibertal (2018-2024)")

In [ ]:
# Create temporal comparison for Perimeter regions
fig, ax = plt.subplots(figsize=(20, 9))

ax.tick_params(axis='x', rotation=45, labelsize=14)
ax.tick_params(axis='y', labelsize=14)